In [ ]:
# отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

# будем отображать графики прямо в jupyter'e
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 

#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 8, 5
import pandas as pd
# Standard python libraries
import logging
import os
import time
import requests
logging.basicConfig(format='[%(asctime)s] (%(levelname)s): %(message)s', level=logging.INFO)
import lightautoml

# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import torch

# Imports from our package
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task

import multiprocessing as mp

# set option of max 100 displayed rows
pd.set_option('display.max_rows', 200)

In [ ]:
import numpy as np 
import pandas as pd

In [ ]:
konh = pd.read_csv('/content/konh.csv')

,id_client,id_contract,IP_flag,F1100,F1110,F1150,F1160,F1170,F1180,F1190,F1200,F1210,F1220,F1230,F1240,F1250,F1260,F1300,F1310,F1320,F1350,F1360,F1370,F1400,F1410,F1420,F1450,F1500,F1510,F1520,F1530,F1550,F1600,F1700,F2100,F2110,F2120,F2200,F2210,F2220,...,new4,new5,new6,new7,new8,new9,new10,new11,new12,new13,new14,new15,new16,new17,new18,new19,new20,new21,new22,new23,new24,new25,new26,new27,new28,new_F1150,new_F1230,new_F1410,new_F1510,new_F1520,new_F2110,new_F2120,new_F2200,new_F2210,new_F2220,new_F2300,new_F2320,new_F2330,new_F2400,TARGET
0,1,5166.0,0.0,10000.0,NaN,10000.0,NaN,NaN,NaN,NaN,7035000.0,NaN,NaN,NaN,NaN,7035000.0,NaN,6121000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,924000.0,NaN,924000.0,NaN,NaN,7045000.0,7045000.0,NaN,32078000.0,27921000.0,4157000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.125000,0.550000,2.226551e+06,0.041322,0.458678,1.159381e+06,NaN,NaN,NaN,NaN,NaN,6.050000,2.00,5.045455,9.000000,4.686366,0.0,NaN,NaN,NaN,38000.0,5472000.0,4554000.0,918000.0,NaN,NaN,918000.0,NaN,NaN,918000.0,1.0
1,2,2389.0,0.0,14626000.0,NaN,14414000.0,NaN,212000.0,NaN,NaN,14939000.0,8521000.0,NaN,6395000.0,NaN,23000.0,NaN,18901000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10664000.0,8995000.0,1669000.0,NaN,NaN,29565000.0,29565000.0,NaN,28874000.0,26330000.0,2544000.0,NaN,NaN,...,NaN,1.0,inf,NaN,NaN,NaN,NaN,NaN,NaN,0.047619,0.619048,3.181044e+06,0.022727,0.465909,4.343591e+06,NaN,NaN,NaN,NaN,NaN,4.190476,2.00,3.153846,3.153846,4.306454,-1180000.0,6213000.0,NaN,8525000.0,19000.0,17403000.0,16235000.0,1168000.0,NaN,NaN,1368000.0,NaN,-200000.0,1095000.0,0.0
2,3,17329.0,0.0,3745000.0,NaN,3745000.0,NaN,NaN,NaN,NaN,9131000.0,888000.0,67000.0,6261000.0,NaN,645000.0,1270000.0,40000.0,40000.0,NaN,NaN,NaN,NaN,10113000.0,10113000.0,NaN,NaN,2723000.0,NaN,2723000.0,NaN,NaN,12876000.0,12876000.0,120000.0,378000.0,258000.0,73000.0,2000.0,45000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,NaN,NaN,NaN,NaN,1005000.0,-2225000.0,-2391000.0,NaN,2440000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,5,15086.0,0.0,87239000.0,NaN,86087000.0,NaN,NaN,262000.0,890000.0,75181000.0,23412000.0,NaN,29428000.0,10200000.0,12141000.0,NaN,134209000.0,10518000.0,NaN,NaN,NaN,104475000.0,NaN,NaN,NaN,NaN,28211000.0,NaN,28111000.0,NaN,100000.0,162420000.0,162420000.0,104340000.0,871384000.0,767044000.0,40236000.0,64104000.0,NaN,...,2.0,6.0,10.0,0.0,1.000000,3.383690e+05,0.000000,0.863636,2.336221e+05,0.003247,0.938312,1.948469e+05,0.001608,0.919614,2.401059e+05,22.000000,NaN,19.0,17.000000,11.737408,6.058442,3.00,5.937716,5.023102,6.189865,9112000.0,2017000.0,NaN,NaN,-1121000.0,23631000.0,-7613000.0,24452000.0,6792000.0,NaN,23302000.0,256000.0,NaN,19022000.0,0.0
4,7,933.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,inf,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8917,12731,10258.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.555556,9.865797e+05,0.050000,0.650000,9.207681e+05,0.011628,0.610465,1.342066e+05,0.013193,0.620053,1.631243e+05,2.222222,inf,2.6,1.333333,1.244391,1.101744,1.25,1.119048,1.126316,1.369004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
8918,12732,10005.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [ ]:
l_d = pd.read_csv('/content/last_df_mkd(5.11).csv')

In [ ]:
l_d = l_d.drop(['Unnamed: 0'],axis=1)

In [ ]:
konh = konh.drop(['Unnamed: 0'],axis=1)

In [ ]:
obj_df = l_d.loc[:,['id_client','IP_flag','EGRPOINCLUDED','OKFS_GROUP','OKOPF_GROUP','OKOGU_GROUP','OKATO_REGIONCODE','OKATO_FED','OKTMO_CODE_fixed','OKTMO_FED','WORKERSRANGE','OKVED_CODE_fixed','FLAG_DISQUALIFICATION','SEX_NAME','CITIZENSHIP_NAME','TELEPHONECOUNT_fixed','MANAGERCOUNTINCOUNTRY_fixed','MANAGERCOUNTINREGION_fixed','MANAGERINNCOUNT_fixed']]

In [ ]:
obj_df = obj_df.astype('category')

In [ ]:
id = konh.id_client.array

In [ ]:
new_obj_df = pd.DataFrame(columns=obj_df.columns)

In [ ]:
for x in id:
    new_obj_df = new_obj_df.append(obj_df[obj_df.id_client==x][0:1])


In [ ]:
obj_df.sort_values(by='id_client')

,id_client,IP_flag,EGRPOINCLUDED,OKFS_GROUP,OKOPF_GROUP,OKOGU_GROUP,OKATO_REGIONCODE,OKATO_FED,OKTMO_CODE_fixed,OKTMO_FED,WORKERSRANGE,OKVED_CODE_fixed,FLAG_DISQUALIFICATION,SEX_NAME,CITIZENSHIP_NAME,TELEPHONECOUNT_fixed,MANAGERCOUNTINCOUNTRY_fixed,MANAGERCOUNTINREGION_fixed,MANAGERINNCOUNT_fixed
5165,1,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,6 .. 10,43.11,NaN,NaN,NaN,1.0,1.0,1.0,1.0
2388,2,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,6 .. 10,31.09,NaN,NaN,NaN,2.0,1.0,1.0,1.0
17328,3,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,0 .. 5,10.11,NaN,NaN,NaN,2.0,1.0,1.0,1.0
19083,4,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,79701000001.0,40.0,51 .. 100,18.12,NaN,NaN,NaN,1.0,3.0,1.0,1.0
15085,5,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,79701000001.0,40.0,251 .. 500,10.51,NaN,NaN,NaN,2.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10257,12731,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,55.0,NaN,RARE_VALUE,NaN,0 .. 5,45.31.1,NaN,NaN,NaN,3.0,3.0,3.0,3.0
10004,12732,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,55.0,NaN,RARE_VALUE,NaN,0 .. 5,68.20,NaN,NaN,NaN,3.0,3.0,3.0,3.0
5290,12733,0,1.0,Иностранная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,55.0,NaN,RARE_VALUE,NaN,11 .. 15,46.71,NaN,NaN,NaN,2.0,2.0,2.0,2.0
3303,12734,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,55.0,NaN,RARE_VALUE,NaN,0 .. 5,RARE_VALUE,NaN,NaN,NaN,1.0,2.0,1.0,2.0


In [ ]:
konh  = konh.drop(['OKTMO_CODE'],axis=1)

In [ ]:
last_train = pd.merge(konh, new_obj_df, on='id_client', how='outer')

In [ ]:
last_train = last_train.drop(['IP_flag_x'],axis=1)

In [ ]:
last_train

,id_client,id_contract,F1100,F1110,F1150,F1160,F1170,F1180,F1190,F1200,F1210,F1220,F1230,F1240,F1250,F1260,F1300,F1310,F1320,F1350,F1360,F1370,F1400,F1410,F1420,F1450,F1500,F1510,F1520,F1530,F1550,F1600,F1700,F2100,F2110,F2120,F2200,F2210,F2220,F2300,...,new22,new23,new24,new25,new26,new27,new28,new_F1150,new_F1230,new_F1410,new_F1510,new_F1520,new_F2110,new_F2120,new_F2200,new_F2210,new_F2220,new_F2300,new_F2320,new_F2330,new_F2400,TARGET,IP_flag_y,EGRPOINCLUDED_y,OKFS_GROUP,OKOPF_GROUP,OKOGU_GROUP,OKATO_REGIONCODE_y,OKATO_FED_y,OKTMO_CODE_fixed,OKTMO_FED_y,WORKERSRANGE,OKVED_CODE_fixed,FLAG_DISQUALIFICATION_y,SEX_NAME,CITIZENSHIP_NAME,TELEPHONECOUNT_fixed,MANAGERCOUNTINCOUNTRY_fixed,MANAGERCOUNTINREGION_fixed,MANAGERINNCOUNT_fixed
0,1,5166.0,10000.0,NaN,10000.0,NaN,NaN,NaN,NaN,7035000.0,NaN,NaN,NaN,NaN,7035000.0,NaN,6121000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,924000.0,NaN,924000.0,NaN,NaN,7045000.0,7045000.0,NaN,32078000.0,27921000.0,4157000.0,NaN,NaN,4157000.0,...,NaN,NaN,6.050000,2.00,5.045455,9.000000,4.686366,0.0,NaN,NaN,NaN,38000.0,5472000.0,4554000.0,918000.0,NaN,NaN,918000.0,NaN,NaN,918000.0,1.0,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,6 .. 10,43.11,NaN,NaN,NaN,1.0,1.0,1.0,1.0
1,2,2389.0,14626000.0,NaN,14414000.0,NaN,212000.0,NaN,NaN,14939000.0,8521000.0,NaN,6395000.0,NaN,23000.0,NaN,18901000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10664000.0,8995000.0,1669000.0,NaN,NaN,29565000.0,29565000.0,NaN,28874000.0,26330000.0,2544000.0,NaN,NaN,2126000.0,...,NaN,NaN,4.190476,2.00,3.153846,3.153846,4.306454,-1180000.0,6213000.0,NaN,8525000.0,19000.0,17403000.0,16235000.0,1168000.0,NaN,NaN,1368000.0,NaN,-200000.0,1095000.0,0.0,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,6 .. 10,31.09,NaN,NaN,NaN,2.0,1.0,1.0,1.0
2,3,17329.0,3745000.0,NaN,3745000.0,NaN,NaN,NaN,NaN,9131000.0,888000.0,67000.0,6261000.0,NaN,645000.0,1270000.0,40000.0,40000.0,NaN,NaN,NaN,NaN,10113000.0,10113000.0,NaN,NaN,2723000.0,NaN,2723000.0,NaN,NaN,12876000.0,12876000.0,120000.0,378000.0,258000.0,73000.0,2000.0,45000.0,-1000.0,...,NaN,NaN,2.000000,NaN,NaN,NaN,NaN,1005000.0,-2225000.0,-2391000.0,NaN,2440000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,0 .. 5,10.11,NaN,NaN,NaN,2.0,1.0,1.0,1.0
3,5,15086.0,87239000.0,NaN,86087000.0,NaN,NaN,262000.0,890000.0,75181000.0,23412000.0,NaN,29428000.0,10200000.0,12141000.0,NaN,134209000.0,10518000.0,NaN,NaN,NaN,104475000.0,NaN,NaN,NaN,NaN,28211000.0,NaN,28111000.0,NaN,100000.0,162420000.0,162420000.0,104340000.0,871384000.0,767044000.0,40236000.0,64104000.0,NaN,26446000.0,...,17.000000,11.737408,6.058442,3.00,5.937716,5.023102,6.189865,9112000.0,2017000.0,NaN,NaN,-1121000.0,23631000.0,-7613000.0,24452000.0,6792000.0,NaN,23302000.0,256000.0,NaN,19022000.0,0.0,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,79701000001.0,40.0,251 .. 500,10.51,NaN,NaN,NaN,2.0,1.0,1.0,1.0
4,7,933.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1,NaN,NaN,NaN,NaN,79.0,40.0,79701000001.0,40.0,NaN,47.52.7,NaN,мужской,Российская Федерация,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8917,12731,10258.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [ ]:
last_train = last_train.drop(['FLAG_DISQUALIFICATION_x','EGRPOINCLUDED_x','OKATO_REGIONCODE_x','OKATO_FED_x','OKTMO_FED_x'],axis=1)

In [ ]:
new_obj_df

,id_client,IP_flag,EGRPOINCLUDED,OKFS_GROUP,OKOPF_GROUP,OKOGU_GROUP,OKATO_REGIONCODE,OKATO_FED,OKTMO_CODE_fixed,OKTMO_FED,WORKERSRANGE,OKVED_CODE_fixed,FLAG_DISQUALIFICATION,SEX_NAME,CITIZENSHIP_NAME,TELEPHONECOUNT_fixed,MANAGERCOUNTINCOUNTRY_fixed,MANAGERCOUNTINREGION_fixed,MANAGERINNCOUNT_fixed
5165,1,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,6 .. 10,43.11,NaN,NaN,NaN,1.0,1.0,1.0,1.0
2388,2,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,6 .. 10,31.09,NaN,NaN,NaN,2.0,1.0,1.0,1.0
17328,3,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,0 .. 5,10.11,NaN,NaN,NaN,2.0,1.0,1.0,1.0
15085,5,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,79701000001.0,40.0,251 .. 500,10.51,NaN,NaN,NaN,2.0,1.0,1.0,1.0
932,7,1,NaN,NaN,NaN,NaN,79.0,40.0,79701000001.0,40.0,NaN,47.52.7,NaN,мужской,Российская Федерация,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10257,12731,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,55.0,NaN,RARE_VALUE,NaN,0 .. 5,45.31.1,NaN,NaN,NaN,3.0,3.0,3.0,3.0
10004,12732,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,55.0,NaN,RARE_VALUE,NaN,0 .. 5,68.20,NaN,NaN,NaN,3.0,3.0,3.0,3.0
5290,12733,0,1.0,Иностранная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,55.0,NaN,RARE_VALUE,NaN,11 .. 15,46.71,NaN,NaN,NaN,2.0,2.0,2.0,2.0
3303,12734,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,55.0,NaN,RARE_VALUE,NaN,0 .. 5,RARE_VALUE,NaN,NaN,NaN,1.0,2.0,1.0,2.0


In [ ]:
sorted(list(last_train.new4.unique()))[-2]
l = [4,5,6,7,10,13,16,19,20,21,22,23,24,25,26,27,28]
def min_inf(x):
    if x==np.inf:
        return m+100
    return x
for x in l:
    m = sorted(list(last_train['new'+str(x)].unique()))[-2]
    last_train['new'+str(x)] = last_train['new'+str(x)].apply(min_inf)


In [ ]:
test

NameError: ignored

In [ ]:
last_train['new3'].describe()

count    1.853000e+03
mean     9.178187e+05
std      5.079952e+06
min      0.000000e+00
25%      2.086000e+03
50%      6.148100e+04
75%      3.453702e+05
max      1.122622e+08
Name: new3, dtype: float64

In [ ]:
l_d['new0'].describe()

count    3.615000e+03
mean     1.730700e+06
std      7.173170e+06
min      0.000000e+00
25%      9.400375e+04
50%      1.368927e+05
75%      5.496305e+05
max      1.181978e+08
Name: new0, dtype: float64

In [ ]:
last_train

,id_client,id_contract,F1100,F1110,F1150,F1160,F1170,F1180,F1190,F1200,F1210,F1220,F1230,F1240,F1250,F1260,F1300,F1310,F1320,F1350,F1360,F1370,F1400,F1410,F1420,F1450,F1500,F1510,F1520,F1530,F1550,F1600,F1700,F2100,F2110,F2120,F2200,F2210,F2220,F2300,...,new22,new23,new24,new25,new26,new27,new28,new_F1150,new_F1230,new_F1410,new_F1510,new_F1520,new_F2110,new_F2120,new_F2200,new_F2210,new_F2220,new_F2300,new_F2320,new_F2330,new_F2400,TARGET,IP_flag_y,EGRPOINCLUDED_y,OKFS_GROUP,OKOPF_GROUP,OKOGU_GROUP,OKATO_REGIONCODE_y,OKATO_FED_y,OKTMO_CODE_fixed,OKTMO_FED_y,WORKERSRANGE,OKVED_CODE_fixed,FLAG_DISQUALIFICATION_y,SEX_NAME,CITIZENSHIP_NAME,TELEPHONECOUNT_fixed,MANAGERCOUNTINCOUNTRY_fixed,MANAGERCOUNTINREGION_fixed,MANAGERINNCOUNT_fixed
0,1,5166.0,10000.0,NaN,10000.0,NaN,NaN,NaN,NaN,7035000.0,NaN,NaN,NaN,NaN,7035000.0,NaN,6121000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,924000.0,NaN,924000.0,NaN,NaN,7045000.0,7045000.0,NaN,32078000.0,27921000.0,4157000.0,NaN,NaN,4157000.0,...,NaN,NaN,6.050000,2.00,5.045455,9.000000,4.686366,0.0,NaN,NaN,NaN,38000.0,5472000.0,4554000.0,918000.0,NaN,NaN,918000.0,NaN,NaN,918000.0,1.0,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,6 .. 10,43.11,NaN,NaN,NaN,1.0,1.0,1.0,1.0
1,2,2389.0,14626000.0,NaN,14414000.0,NaN,212000.0,NaN,NaN,14939000.0,8521000.0,NaN,6395000.0,NaN,23000.0,NaN,18901000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10664000.0,8995000.0,1669000.0,NaN,NaN,29565000.0,29565000.0,NaN,28874000.0,26330000.0,2544000.0,NaN,NaN,2126000.0,...,NaN,NaN,4.190476,2.00,3.153846,3.153846,4.306454,-1180000.0,6213000.0,NaN,8525000.0,19000.0,17403000.0,16235000.0,1168000.0,NaN,NaN,1368000.0,NaN,-200000.0,1095000.0,0.0,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,6 .. 10,31.09,NaN,NaN,NaN,2.0,1.0,1.0,1.0
2,3,17329.0,3745000.0,NaN,3745000.0,NaN,NaN,NaN,NaN,9131000.0,888000.0,67000.0,6261000.0,NaN,645000.0,1270000.0,40000.0,40000.0,NaN,NaN,NaN,NaN,10113000.0,10113000.0,NaN,NaN,2723000.0,NaN,2723000.0,NaN,NaN,12876000.0,12876000.0,120000.0,378000.0,258000.0,73000.0,2000.0,45000.0,-1000.0,...,NaN,NaN,2.000000,NaN,NaN,NaN,NaN,1005000.0,-2225000.0,-2391000.0,NaN,2440000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,0 .. 5,10.11,NaN,NaN,NaN,2.0,1.0,1.0,1.0
3,5,15086.0,87239000.0,NaN,86087000.0,NaN,NaN,262000.0,890000.0,75181000.0,23412000.0,NaN,29428000.0,10200000.0,12141000.0,NaN,134209000.0,10518000.0,NaN,NaN,NaN,104475000.0,NaN,NaN,NaN,NaN,28211000.0,NaN,28111000.0,NaN,100000.0,162420000.0,162420000.0,104340000.0,871384000.0,767044000.0,40236000.0,64104000.0,NaN,26446000.0,...,17.000000,11.737408,6.058442,3.00,5.937716,5.023102,6.189865,9112000.0,2017000.0,NaN,NaN,-1121000.0,23631000.0,-7613000.0,24452000.0,6792000.0,NaN,23302000.0,256000.0,NaN,19022000.0,0.0,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,79701000001.0,40.0,251 .. 500,10.51,NaN,NaN,NaN,2.0,1.0,1.0,1.0
4,7,933.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1,NaN,NaN,NaN,NaN,79.0,40.0,79701000001.0,40.0,NaN,47.52.7,NaN,мужской,Российская Федерация,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8917,12731,10258.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [ ]:
new = 'new_F1150	new_F1230	new_F1410	new_F1510	new_F1520	new_F2110	new_F2120	new_F2200	new_F2210	new_F2220	new_F2300	new_F2320	new_F2330'
new = new.split()
new = new + ['new0','new1','new2','new3','new9','new12','new15','new18']

In [ ]:
for x in new:
    m = sorted(list(last_train[x].unique()))[-2]
    last_train[x] = last_train[x].apply(min_inf)

In [ ]:
last_train.describe()

,id_contract,F1100,F1110,F1150,F1160,F1170,F1180,F1190,F1200,F1210,F1220,F1230,F1240,F1250,F1260,F1300,F1310,F1320,F1350,F1360,F1370,F1400,F1410,F1420,F1450,F1500,F1510,F1520,F1530,F1550,F1600,F1700,F2100,F2110,F2120,F2200,F2210,F2220,F2300,F2310,...,new4,new5,new6,new7,new8,new9,new10,new11,new12,new13,new14,new15,new16,new17,new18,new19,new20,new21,new22,new23,new24,new25,new26,new27,new28,new_F1150,new_F1230,new_F1410,new_F1510,new_F1520,new_F2110,new_F2120,new_F2200,new_F2210,new_F2220,new_F2300,new_F2320,new_F2330,new_F2400,TARGET
count,8922.000000,3.813000e+03,5.390000e+02,3.618000e+03,1.240000e+02,7.180000e+02,1.029000e+03,6.870000e+02,6.238000e+03,5.255000e+03,1.483000e+03,6.002000e+03,1.243000e+03,6.101000e+03,1.975000e+03,6.240000e+03,3.778000e+03,3.600000e+01,5.560000e+02,6.410000e+02,3.749000e+03,2.127000e+03,1.312000e+03,9.900000e+02,3.780000e+02,6.083000e+03,2.587000e+03,6.016000e+03,4.560000e+02,4.760000e+02,6.255000e+03,6.254000e+03,3.804000e+03,6.104000e+03,5.933000e+03,6.117000e+03,1.842000e+03,1.584000e+03,6.128000e+03,1.060000e+02,...,828.000000,1315.000000,1492.000000,2253.000000,2204.000000,1.823000e+03,2791.000000,2737.000000,2.336000e+03,3608.000000,3599.000000,3.402000e+03,3753.000000,3749.000000,3.589000e+03,2253.000000,1255.000000,2075.000000,1974.000000,1972.000000,3611.000000,2218.000000,3533.000000,3464.000000,3461.000000,3.144000e+03,5.230000e+03,8.220000e+02,1.770000e+03,5.231000e+03,5.287000e+03,5.116000e+03,5.303000e+03,1.536000e+03,1.208000e+03,5.324000e+03,1.030000e+03,1.241000e+03,5.328000e+03,8922.000000
mean,8927.420745,1.500905e+09,6.935237e+07,1.060895e+09,6.227641e+07,1.455451e+09,8.564511e+07,7.738869e+08,4.514681e+08,8.781025e+07,1.437572e+07,2.746508e+08,2.235322e+08,5.912421e+07,2.321999e+07,6.457392e+08,4.102052e+08,-4.440332e+07,1.318131e+09,5.973565e+07,2.541220e+08,8.580067e+08,1.110346e+09,1.869001e+08,3.580622e+08,4.414704e+08,2.593900e+08,3.033594e+08,1.524438e+08,7.371261e+07,1.364650e+09,1.365276e+09,4.936527e+08,1.347639e+09,1.068106e+09,8.740586e+07,6.287075e+08,1.259131e+08,8.280170e+07,4.010269e+08,...,66.023012,52.398556,33.130693,2.567441,0.628502,2.003402e+06,2.197177,0.625971,1.945206e+06,0.325049,0.612714,1.809537e+06,0.174565,0.606056,1.759142e+06,7.114980,57.970627,15.976995,16.228886,177.834909,7.447513,68.884507,22.333758,16.989016,87.629862,5.279166e+07,1.709445e+07,9.222022e+07,3.161784e+07,1.690577e+07,1.343356e+08,1.112911e+08,5.207869e+06,5.993611e+07,1.559127e+07,2.901361e+06,-8.492533e+05,-4.948053e+06,-3.212004e+05,0.209786
std,4881.111898,2.494452e+10,6.422873e+08,1.698762e+10,2.631264e+08,1.343161e+10,5.648259e+08,1.127785e+10,3.965542e+09,9.081734e+08,1.297399e+08,2.495538e+09,1.856772e+09,7.509933e+08,4.124585e+08,1.472846e+10,1.111992e+10,1.491933e+08,7.990794e+09,7.846531e+08,4.443357e+09,1.238759e+10,1.339969e+10,2.008815e+09,1.761483e+09,4.088649e+09,2.657827e+09,2.543953e+09,5.972178e+08,8.306361e+08,2.283372e+10,2.283563e+10,5.705677e+09,1.087690e+10,7.715642e+09,1.851821e+09,6.120271e+09,1.090643e+09,1.943175e+09,2.906663e+09,...,71.635720,94.222356,69.137279,15.752301,0.350524,1.290116e+07,14.540538,0.326155,8.084966e+06,5.491525,0.300722,1.153111e+07,3.731052,0.275773,1.360440e+07,23.769449,85.473594,46.329588,43.767608,567.264443,43.580584,105.868447,135.570288,88.507205,592.250837,1.286358e+09,3.551110e+08,1.854702e+09,1.015829e+09,4.007758e+08,1.144258e+09,8.639862e+08,6.034318e+08,8.888367e+08,1.075308e+08,1.206665e+09,1.053921e+08,1.904253e+08,1.133635e+09,0.402818
min,2.000000,-1.820000e+05,1.000000e+03,1.000000e+03,-1.175000e+06,1.000000e+03,-2.540000e+05,1.000000e+03,-2.950000e+05,-2.100000e+05,-3.990000e+05,1.000000e+03,-5.200000e+04,-2.103100e+07,-3.701000e+06,-7.865320e+10,1.000000e+03,-8.538600e+08,-2.273630e+08,1.000000e+03,-7.865321e+10,-4.140000e+06,-4.140000e+06,-7.150000e+05,-2.427000e+06,-3.519000e+06,-1.998900e+07,-1.022000e+06,-5.240000e+05,-8.498100e+07,-2.950000e+05,-2.950000e+05,-6.795270e+09,3.000000e+03,-1.110870e+

In [ ]:
last_train.TARGET.value_counts()

0.000000    6977
1.000000    1791
0.500000      64
0.333333      20
0.666667      19
0.750000       9
0.250000       7
0.875000       4
0.800000       3
0.166667       3
0.200000       3
0.900000       2
0.888889       2
0.125000       1
0.026667       1
0.090909       1
0.927273       1
0.939394       1
0.285714       1
0.909091       1
0.100000       1
0.933333       1
0.937500       1
0.969697       1
0.700000       1
0.833333       1
0.058824       1
0.003413       1
0.600000       1
0.941176       1
0.916667       1
Name: TARGET, dtype: int64

In [ ]:
konh.TARGET

0       1.0
1       0.0
2       1.0
3       0.0
4       0.0
       ... 
8917    1.0
8918    0.0
8919    0.0
8920    1.0
8921    0.0
Name: TARGET, Length: 8922, dtype: float64

In [ ]:
import scipy.stats as st

In [ ]:
le_ri = []
def per_fil(x):
    if x<l:
        return l
    if x>r:
        return r
    return x
for x in last_train.columns[2:162]:
    data = last_train[x]
    l,r = st.t.interval(alpha=0.99, df=len(data)-1, loc=np.mean(data), scale=np.std(data, ddof=1) / np.sqrt(np.size(data)))
    le_ri.append(l)
    le_ri.append(r)
    last_train[x] = last_train[x].apply(per_fil)


In [ ]:
last_train.iloc[:,2:162]

,F1100,F1110,F1150,F1160,F1170,F1180,F1190,F1200,F1210,F1220,F1230,F1240,F1250,F1260,F1300,F1310,F1320,F1350,F1360,F1370,F1400,F1410,F1420,F1450,F1500,F1510,F1520,F1530,F1550,F1600,F1700,F2100,F2110,F2120,F2200,F2210,F2220,F2300,F2310,F2320,...,new3,new4,new5,new6,new7,new8,new9,new10,new11,new12,new13,new14,new15,new16,new17,new18,new19,new20,new21,new22,new23,new24,new25,new26,new27,new28,new_F1150,new_F1230,new_F1410,new_F1510,new_F1520,new_F2110,new_F2120,new_F2200,new_F2210,new_F2220,new_F2300,new_F2320,new_F2330,new_F2400
0,8.205209e+08,NaN,5.975429e+08,NaN,NaN,NaN,NaN,3.433044e+08,NaN,NaN,NaN,NaN,3.864020e+07,NaN,2.440075e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.299489e+08,NaN,2.339708e+08,NaN,NaN,7.418399e+08,7.424142e+08,NaN,1.050962e+09,8.576547e+08,3.689578e+07,NaN,NaN,2.979987e+07,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.175263,0.604512,2.124058e+06,0.072797,0.598534,1.388070e+06,NaN,NaN,NaN,NaN,NaN,6.258814,65.996851,18.635958,14.574904,71.475694,1.770512e+07,NaN,NaN,NaN,5.974256e+06,1.031249e+08,8.772515e+07,9.180000e+05,NaN,NaN,918000.0,NaN,NaN,918000.0
1,8.205209e+08,NaN,5.975429e+08,NaN,1.089092e+09,NaN,NaN,3.433044e+08,6.303902e+07,NaN,2.065828e+08,NaN,3.864020e+07,NaN,2.440075e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.299489e+08,1.868954e+08,2.339708e+08,NaN,NaN,7.418399e+08,7.424142e+08,NaN,1.050962e+09,8.576547e+08,3.689578e+07,NaN,NaN,2.979987e+07,NaN,NaN,...,779258.492949,NaN,49.828558,35.016473,NaN,NaN,NaN,NaN,NaN,NaN,0.175263,0.619048,2.124058e+06,0.072797,0.598534,2.130214e+06,NaN,NaN,NaN,NaN,NaN,6.258814,65.996851,18.635958,14.574904,71.475694,1.770512e+07,7.408478e+06,NaN,8525000.0,5.974256e+06,1.031249e+08,8.772515e+07,1.168000e+06,NaN,NaN,1368000.0,NaN,-200000.0,1095000.0
2,8.205209e+08,NaN,5.975429e+08,NaN,NaN,NaN,NaN,3.433044e+08,6.303902e+07,1.083694e+07,2.065828e+08,NaN,3.864020e+07,1.196981e+07,2.440075e+08,1.068996e+08,NaN,NaN,NaN,NaN,5.201243e+08,7.448575e+08,NaN,NaN,3.299489e+08,NaN,2.339708e+08,NaN,NaN,7.418399e+08,7.424142e+08,3.380253e+08,1.050962e+09,8.576547e+08,3.689578e+07,4.617717e+08,9.616489e+07,2.979987e+07,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.175263,0.604512,NaN,0.072797,0.598534,NaN,NaN,NaN,NaN,NaN,NaN,6.258814,NaN,NaN,NaN,NaN,1.770512e+07,7.408478e+06,4.163157e+07,NaN,5.974256e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8.205209e+08,NaN,5.975429e+08,NaN,NaN,7.023898e+07,4.662736e+08,3.433044e+08,6.303902e+07,NaN,2.065828e+08,1.728871e+08,3.864020e+07,NaN,2.440075e+08,1.068996e+08,NaN,NaN,NaN,1.329255e+08,NaN,NaN,NaN,NaN,3.299489e+08,NaN,2.339708e+08,NaN,5.105627e+07,7.418399e+08,7.424142e+08,3.380253e+08,1.050962e+09,8.576547e+08,4.023600e+07,4.617717e+08,NaN,2.979987e+07,NaN,3.198422e+07,...,779258.492949,64.069084,49.828558,31.244912,2.137783,0.638063,1.651511e+06,1.800571,0.634868,1.724681e+06,0.175263,0.620917,1.495016e+06,0.072797,0.613578,1.388070e+06,7.763312,NaN,17.240675,17.000000,162.362268,6.258814,65.996851,18.635958,14.574904,71.475694,1.770512e+07,7.408478e+06,NaN,NaN,5.974256e+06,1.031249e+08,8.772515e+07,2.166701e+07,3.569230e+07,NaN,23302000.0,256000.0,NaN,19022000.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,779258.492949,NaN,54.968554,31.244912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8917,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.137783,0.618941,1

In [ ]:
last_train['random'] = [(10**6)*random.random() for i in range(8922)]

# BLYT ETO PIZDEC

In [ ]:
all_df = pd.read_csv('/content/last_df_mkd(5.11).csv')

In [ ]:
all_df = all_df.drop(['Unnamed: 0'],axis=1)

In [ ]:
konh_test = all_df.iloc[17891:,:]

In [ ]:
konh_test = konh_test.groupby('id_client').aggregate('mean').reset_index()

In [ ]:
konh_test.F1160.value_counts()

2.150000e+05    1
8.009000e+06    1
3.687500e+07    1
8.710000e+05    1
3.025000e+06    1
1.262300e+07    1
6.620900e+07    1
1.266000e+06    1
1.184000e+06    1
3.908000e+06    1
1.331000e+06    1
2.155800e+07    1
1.900000e+04    1
7.746000e+06    1
5.930000e+05    1
1.150000e+06    1
5.451000e+06    1
1.133000e+06    1
8.940000e+05    1
4.000000e+04    1
1.037000e+07    1
1.259800e+07    1
3.832400e+07    1
3.690000e+05    1
7.280000e+06    1
3.312000e+06    1
9.790400e+07    1
1.318000e+06    1
9.172000e+06    1
1.693980e+08    1
6.552000e+06    1
1.300000e+05    1
1.330000e+05    1
1.309400e+07    1
2.230000e+05    1
1.390000e+05    1
5.692000e+06    1
1.015570e+08    1
2.977000e+06    1
2.820000e+05    1
2.649000e+06    1
6.000000e+03    1
2.056000e+06    1
2.351000e+06    1
8.238000e+06    1
7.189000e+06    1
2.337998e+09    1
3.823000e+06    1
3.031100e+07    1
3.592000e+06    1
4.436000e+07    1
6.030000e+05    1
9.507000e+06    1
4.178000e+06    1
3.998000e+06    1
Name: F116

In [ ]:
konh_test.describe()

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:3968: RuntimeWarning: invalid value encountered in multiply
  x2 = take(ap, indices_above, axis=axis) * weights_above


,id_client,id_contract,IP_flag,F1100,F1110,F1150,F1160,F1170,F1180,F1190,F1200,F1210,F1220,F1230,F1240,F1250,F1260,F1300,F1310,F1320,F1350,F1360,F1370,F1400,F1410,F1420,F1450,F1500,F1510,F1520,F1530,F1550,F1600,F1700,F2100,F2110,F2120,F2200,F2210,F2220,...,new3,new4,new5,new6,new7,new8,new9,new10,new11,new12,new13,new14,new15,new16,new17,new18,new19,new20,new21,new22,new23,new24,new25,new26,new27,new28,new_F1150,new_F1230,new_F1410,new_F1510,new_F1520,new_F2110,new_F2120,new_F2200,new_F2210,new_F2220,new_F2300,new_F2320,new_F2330,new_F2400
count,4761.000000,4761.000000,4761.000000,1.797000e+03,2.720000e+02,1.697000e+03,5.500000e+01,3.100000e+02,3.940000e+02,2.810000e+02,3.145000e+03,2.610000e+03,6.880000e+02,2.987000e+03,5.740000e+02,3.070000e+03,8.980000e+02,3.149000e+03,1.808000e+03,8.000000e+00,2.080000e+02,2.510000e+02,1.772000e+03,9.840000e+02,6.480000e+02,3.850000e+02,1.860000e+02,3.043000e+03,1.361000e+03,3.012000e+03,1.400000e+02,1.780000e+02,3.155000e+03,3.155000e+03,1.800000e+03,3.038000e+03,2.956000e+03,3.058000e+03,1.041000e+03,7.390000e+02,...,9.150000e+02,382.00,636.0,683.000000,935.000000,906.000000,6.330000e+02,1245.000000,1219.000000,9.900000e+02,1581.00000,1578.000000,1.477000e+03,1668.000000,1666.000000,1.587000e+03,931.000000,569.0,852.000000,777.0,776.000000,1583.000000,1057.0,1542.000000,1519.000000,1518.000000,1.424000e+03,2.528000e+03,4.330000e+02,9.310000e+02,2.543000e+03,2.549000e+03,2.481000e+03,2.579000e+03,8.510000e+02,5.400000e+02,2.591000e+03,4.500000e+02,6.130000e+02,2.589000e+03
mean,6411.463558,21653.221380,0.223698,1.224593e+09,8.137971e+07,8.142487e+08,5.665060e+07,1.518055e+09,8.620192e+07,1.997197e+08,4.576624e+08,8.730769e+07,1.774919e+07,2.640738e+08,2.706665e+08,8.184210e+07,8.516107e+06,3.991255e+08,1.376205e+08,-4.263112e+07,2.008668e+09,2.955346e+07,2.070916e+08,1.128145e+09,1.371119e+09,1.506384e+08,2.547871e+08,4.195796e+08,2.152774e+08,2.574578e+08,5.660440e+08,6.697251e+07,1.153679e+09,1.154907e+09,4.496936e+08,1.134184e+09,8.875267e+08,7.621482e+07,6.191760e+08,1.335638e+08,...,9.715129e+05,inf,inf,inf,inf,0.591401,3.705726e+06,inf,0.603054,2.146871e+06,inf,0.565405,2.059353e+06,inf,0.558123,1.542546e+06,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,1.175081e+08,5.383647e+07,5.109119e+08,3.509206e+07,2.095928e+07,1.289572e+08,1.148098e+08,6.015796e+06,2.083355e+08,1.830132e+07,-2.528290e+07,9.130282e+06,6.455460e+06,-2.607749e+07
std,3656.888423,2123.465805,0.416150,1.954608e+10,8.351265e+08,1.022968e+10,3.147659e+08,1.683036e+10,4.057336e+08,1.314710e+09,4.994833e+09,6.426732e+08,1.937430e+08,3.230437e+09,2.283911e+09,1.428569e+09,5.692985e+07,6.066536e+09,1.849906e+09,1.025508e+08,1.208077e+10,1.468697e+08,3.249858e+09,1.846171e+10,1.775634e+10,1.144549e+09,9.112902e+08,4.748172e+09,2.532616e+09,2.342308e+09,4.179752e+09,4.316533e+08,1.837766e+10,1.837797e+10,5.136588e+09,9.801854e+09,7.140327e+09,1.905805e+09,6.329772e+09,1.255894e+09,...,1.067505e+07,NaN,NaN,NaN,NaN,0.356126,4.121337e+07,NaN,0.332699,8.167606e+06,NaN,0.301812,9.523033e+06,NaN,0.272549,5.863587e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.251187e+09,1.993574e+09,8.568284e+09,2.331736e+09,5.581078e+08,1.026287e+09,1.122583e+09,3.726482e+08,5.633240e+09,1.682358e+08,2.474948e+09,2.218286e+08,2.378461e+08,2.426928e+09
min,4.000000,17893.000000,0.000000,4.740000e+02,1.000000e+03,4.740000e+02,6.000000e+03,1.000000e+03,-1.830000e+05,1.000000e+03,-1.000000e+04,1.000000e+03,2.000000e+01,-7.919000e+06,1.000000e+03,-9.810000e+05,-2.200000e+05,-6.288671e+09,1.000000e+01,-2.956510e+08,-3.417000e+06,-5.030000e+05,-2.015361e+10,-2.750000e+06,-2.750000e+06,-9.700000e+04,-7.120000e+05,-6.193000e+06,-2.500000e+07,-1.903000e+06,-5.240000e+05,1.000000e+03,-1.000000e+04,-1.000000e+04,-2.212800e+10,7.000000e+03,-3.413530e+09,-2.305835e+10,-6.683200e+07,1.000000e+03,...,0.000000e+00,1.00,1.0,1.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.00000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,1.000000,1.0,1

In [ ]:
konh.describe()

,id_client,id_contract,IP_flag,F1100,F1110,F1150,F1160,F1170,F1180,F1190,F1200,F1210,F1220,F1230,F1240,F1250,F1260,F1300,F1310,F1320,F1350,F1360,F1370,F1400,F1410,F1420,F1450,F1500,F1510,F1520,F1530,F1550,F1600,F1700,F2100,F2110,F2120,F2200,F2210,F2220,...,new4,new5,new6,new7,new8,new9,new10,new11,new12,new13,new14,new15,new16,new17,new18,new19,new20,new21,new22,new23,new24,new25,new26,new27,new28,new_F1150,new_F1230,new_F1410,new_F1510,new_F1520,new_F2110,new_F2120,new_F2200,new_F2210,new_F2220,new_F2300,new_F2320,new_F2330,new_F2400,TARGET
count,8922.000000,8922.000000,8922.000000,3.813000e+03,5.390000e+02,3.618000e+03,1.240000e+02,7.180000e+02,1.029000e+03,6.870000e+02,6.238000e+03,5.255000e+03,1.483000e+03,6.002000e+03,1.243000e+03,6.101000e+03,1.975000e+03,6.240000e+03,3.778000e+03,3.600000e+01,5.560000e+02,6.410000e+02,3.749000e+03,2.127000e+03,1.312000e+03,9.900000e+02,3.780000e+02,6.083000e+03,2.587000e+03,6.016000e+03,4.560000e+02,4.760000e+02,6.255000e+03,6.254000e+03,3.804000e+03,6.104000e+03,5.933000e+03,6.117000e+03,1.842000e+03,1.584000e+03,...,828.000000,1315.000000,1492.000000,2253.000000,2204.000000,1.823000e+03,2791.000000,2737.000000,2.336000e+03,3608.000000,3599.000000,3.402000e+03,3753.000000,3749.000000,3.589000e+03,2253.000000,1255.000000,2075.000000,1974.000000,1972.000000,3611.000000,2218.000000,3533.000000,3464.000000,3461.000000,3.144000e+03,5.230000e+03,8.220000e+02,1.770000e+03,5.231000e+03,5.287000e+03,5.116000e+03,5.303000e+03,1.536000e+03,1.208000e+03,5.324000e+03,1.030000e+03,1.241000e+03,5.328000e+03,8922.000000
mean,6367.712845,8927.420745,0.183577,1.500905e+09,6.935237e+07,1.060895e+09,6.227641e+07,1.455451e+09,8.564511e+07,7.738869e+08,4.514681e+08,8.781025e+07,1.437572e+07,2.746508e+08,2.235322e+08,5.912421e+07,2.321999e+07,6.457392e+08,4.102052e+08,-4.440332e+07,1.318131e+09,5.973565e+07,2.541220e+08,8.580067e+08,1.110346e+09,1.869001e+08,3.580622e+08,4.414704e+08,2.593900e+08,3.033594e+08,1.524438e+08,7.371261e+07,1.364650e+09,1.365276e+09,4.936527e+08,1.347639e+09,1.068106e+09,8.740586e+07,6.287075e+08,1.259131e+08,...,inf,inf,inf,inf,0.628502,2.003402e+06,inf,0.625971,1.945206e+06,inf,0.612714,1.809537e+06,inf,0.606056,1.759142e+06,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,5.279166e+07,1.709445e+07,9.222022e+07,3.161784e+07,1.690577e+07,1.343356e+08,1.112911e+08,5.207869e+06,5.993611e+07,1.559127e+07,2.901361e+06,-8.492533e+05,-4.948053e+06,-3.212004e+05,0.209786
std,3680.554333,4881.111898,0.385001,2.494452e+10,6.422873e+08,1.698762e+10,2.631264e+08,1.343161e+10,5.648259e+08,1.127785e+10,3.965542e+09,9.081734e+08,1.297399e+08,2.495538e+09,1.856772e+09,7.509933e+08,4.124585e+08,1.472846e+10,1.111992e+10,1.491933e+08,7.990794e+09,7.846531e+08,4.443357e+09,1.238759e+10,1.339969e+10,2.008815e+09,1.761483e+09,4.088649e+09,2.657827e+09,2.543953e+09,5.972178e+08,8.306361e+08,2.283372e+10,2.283563e+10,5.705677e+09,1.087690e+10,7.715642e+09,1.851821e+09,6.120271e+09,1.090643e+09,...,NaN,NaN,NaN,NaN,0.350524,1.290116e+07,NaN,0.326155,8.084966e+06,NaN,0.300722,1.153111e+07,NaN,0.275773,1.360440e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.286358e+09,3.551110e+08,1.854702e+09,1.015829e+09,4.007758e+08,1.144258e+09,8.639862e+08,6.034318e+08,8.888367e+08,1.075308e+08,1.206665e+09,1.053921e+08,1.904253e+08,1.133635e+09,0.402818
min,1.000000,2.000000,0.000000,-1.820000e+05,1.000000e+03,1.000000e+03,-1.175000e+06,1.000000e+03,-2.540000e+05,1.000000e+03,-2.950000e+05,-2.100000e+05,-3.990000e+05,1.000000e+03,-5.200000e+04,-2.103100e+07,-3.701000e+06,-7.865320e+10,1.000000e+03,-8.538600e+08,-2.273630e+08,1.000000e+03,-7.865321e+10,-4.140000e+06,-4.140000e+06,-7.150000e+05,-2.427000e+06,-3.519000e+06,-1.998900e+07,-1.022000e+06,-5.240000e+05,-8.498100e+07,-2.950000e+05,-2.950000e+05,-6.795270e+09,3.000000e+03,-1.110870e+08,-6.858694e+09,1.000000e+03,5.000000e+02,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.00

In [ ]:
giper_konh = pd.concat([konh,konh_test])

In [ ]:
giper_konh

,id_client,id_contract,IP_flag,F1100,F1110,F1150,F1160,F1170,F1180,F1190,F1200,F1210,F1220,F1230,F1240,F1250,F1260,F1300,F1310,F1320,F1350,F1360,F1370,F1400,F1410,F1420,F1450,F1500,F1510,F1520,F1530,F1550,F1600,F1700,F2100,F2110,F2120,F2200,F2210,F2220,...,new5,new6,new7,new8,new9,new10,new11,new12,new13,new14,new15,new16,new17,new18,new19,new20,new21,new22,new23,new24,new25,new26,new27,new28,new_F1150,new_F1230,new_F1410,new_F1510,new_F1520,new_F2110,new_F2120,new_F2200,new_F2210,new_F2220,new_F2300,new_F2320,new_F2330,new_F2400,TARGET,OKTMO_CODE
0,1,5166.0,0.0,10000.0,NaN,10000.0,NaN,NaN,NaN,NaN,7035000.0,NaN,NaN,NaN,NaN,7035000.0,NaN,6121000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,924000.0,NaN,924000.0,NaN,NaN,7045000.0,7045000.0,NaN,32078000.0,27921000.0,4157000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.125000,0.550000,2.226551e+06,0.041322,0.458678,1.159381e+06,NaN,NaN,NaN,NaN,NaN,6.050000,2.0,5.045455,9.000000,4.686366,0.0,NaN,NaN,NaN,38000.0,5472000.0,4554000.0,918000.0,NaN,NaN,918000.0,NaN,NaN,918000.0,1.0,NaN
1,2,2389.0,0.0,14626000.0,NaN,14414000.0,NaN,212000.0,NaN,NaN,14939000.0,8521000.0,NaN,6395000.0,NaN,23000.0,NaN,18901000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10664000.0,8995000.0,1669000.0,NaN,NaN,29565000.0,29565000.0,NaN,28874000.0,26330000.0,2544000.0,NaN,NaN,...,1.0,inf,NaN,NaN,NaN,NaN,NaN,NaN,0.047619,0.619048,3.181044e+06,0.022727,0.465909,4.343591e+06,NaN,NaN,NaN,NaN,NaN,4.190476,2.0,3.153846,3.153846,4.306454,-1180000.0,6213000.0,NaN,8525000.0,19000.0,17403000.0,16235000.0,1168000.0,NaN,NaN,1368000.0,NaN,-200000.0,1095000.0,0.0,NaN
2,3,17329.0,0.0,3745000.0,NaN,3745000.0,NaN,NaN,NaN,NaN,9131000.0,888000.0,67000.0,6261000.0,NaN,645000.0,1270000.0,40000.0,40000.0,NaN,NaN,NaN,NaN,10113000.0,10113000.0,NaN,NaN,2723000.0,NaN,2723000.0,NaN,NaN,12876000.0,12876000.0,120000.0,378000.0,258000.0,73000.0,2000.0,45000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,NaN,NaN,NaN,NaN,1005000.0,-2225000.0,-2391000.0,NaN,2440000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
3,5,15086.0,0.0,87239000.0,NaN,86087000.0,NaN,NaN,262000.0,890000.0,75181000.0,23412000.0,NaN,29428000.0,10200000.0,12141000.0,NaN,134209000.0,10518000.0,NaN,NaN,NaN,104475000.0,NaN,NaN,NaN,NaN,28211000.0,NaN,28111000.0,NaN,100000.0,162420000.0,162420000.0,104340000.0,871384000.0,767044000.0,40236000.0,64104000.0,NaN,...,6.0,10.0,0.0,1.0,338369.0,0.0,0.863636,233622.058824,0.003247,0.938312,1.948469e+05,0.001608,0.919614,2.401059e+05,22.0,NaN,19.0,17.0,11.737408,6.058442,3.0,5.937716,5.023102,6.189865,9112000.0,2017000.0,NaN,NaN,-1121000.0,23631000.0,-7613000.0,24452000.0,6792000.0,NaN,23302000.0,256000.0,NaN,19022000.0,0.0,NaN
4,7,933.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,inf,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4756,12725,20931.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,135000.0,NaN,3000.0,79000.0,NaN,28000.0,25000.0,17000.0,30000.0,NaN,NaN,NaN,-13000.0,NaN,NaN,NaN,NaN,118000.0,NaN,118000.0,NaN,NaN,135000.0,135000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,NaN,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.532300e+10
4757,12726,21787.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [ ]:
konh = konh.drop(['TARGET'],axis=1)
konh_test = konh_test.drop(['OKTMO_CODE'],axis=1)

In [ ]:
25221-7330

17891

In [ ]:
train_df = all_df.iloc[:17891,:]
test_df = all_df.iloc[17891:,:]

In [ ]:
obj_df_tr = train_df.loc[:,['id_client','IP_flag','EGRPOINCLUDED','OKFS_GROUP','OKOPF_GROUP','OKOGU_GROUP','OKATO_REGIONCODE','OKATO_FED','OKTMO_CODE_fixed','OKTMO_FED','WORKERSRANGE','OKVED_CODE_fixed','FLAG_DISQUALIFICATION','SEX_NAME','CITIZENSHIP_NAME','TELEPHONECOUNT_fixed','MANAGERCOUNTINCOUNTRY_fixed','MANAGERCOUNTINREGION_fixed','MANAGERINNCOUNT_fixed']]

In [ ]:
obj_df_tr = obj_df_tr.astype('category')

In [ ]:
id_tr = konh.id_client.array

In [ ]:
new_obj_df_tr = pd.DataFrame(columns=obj_df_tr.columns)

In [ ]:
for x in id_tr:
    new_obj_df_tr = new_obj_df_tr.append(obj_df_tr[obj_df_tr.id_client==x][0:1])


In [ ]:
id_te

NameError: ignored

In [ ]:
obj_df_te = test_df.loc[:,['id_client','IP_flag','EGRPOINCLUDED','OKFS_GROUP','OKOPF_GROUP','OKOGU_GROUP','OKATO_REGIONCODE','OKATO_FED','OKTMO_CODE_fixed','OKTMO_FED','WORKERSRANGE','OKVED_CODE_fixed','FLAG_DISQUALIFICATION','SEX_NAME','CITIZENSHIP_NAME','TELEPHONECOUNT_fixed','MANAGERCOUNTINCOUNTRY_fixed','MANAGERCOUNTINREGION_fixed','MANAGERINNCOUNT_fixed']]

In [ ]:
obj_df_te = obj_df_te.astype('category')

In [ ]:
id_te = konh_test.id_client.array

In [ ]:
new_obj_df_te = pd.DataFrame(columns=obj_df_te.columns)

In [ ]:
for x in id_te:
    new_obj_df_te = new_obj_df_te.append(obj_df_te[obj_df_te.id_client==x][0:1])


In [ ]:
new_obj_df_te

,id_client,IP_flag,EGRPOINCLUDED,OKFS_GROUP,OKOPF_GROUP,OKOGU_GROUP,OKATO_REGIONCODE,OKATO_FED,OKTMO_CODE_fixed,OKTMO_FED,WORKERSRANGE,OKVED_CODE_fixed,FLAG_DISQUALIFICATION,SEX_NAME,CITIZENSHIP_NAME,TELEPHONECOUNT_fixed,MANAGERCOUNTINCOUNTRY_fixed,MANAGERCOUNTINREGION_fixed,MANAGERINNCOUNT_fixed
19083,4,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,79701000001.0,40.0,51 .. 100,18.12,NaN,NaN,NaN,1.0,3.0,1.0,1.0
24954,6,0,1.0,Государственная собственность,Некоммерческая унитарная организация,Государственная власть в субъектах РФ,79.0,40.0,79701000001.0,40.0,16 .. 50,71.20.61,NaN,NaN,NaN,2.0,2.0,1.0,1.0
21716,8,0,1.0,Государственная собственность,Коммерческая унитарная организация,Государственная власть в субъектах РФ,79.0,40.0,79701000001.0,40.0,51 .. 100,46.46.1,NaN,NaN,NaN,1.0,1.0,1.0,1.0
19202,18,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,79701000001.0,40.0,0 .. 5,46.46,NaN,NaN,NaN,1.0,1.0,1.0,1.0
18779,19,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,79701000001.0,40.0,6 .. 10,47.21,NaN,NaN,NaN,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20930,12725,0,0.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,45.0,30.0,45323000000.0,30.0,0 .. 5,46.73,NaN,NaN,NaN,1.0,1.0,1.0,1.0
21786,12726,0,0.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,45.0,30.0,45327000000.0,30.0,0 .. 5,45.31,NaN,NaN,NaN,RARE_VALUE,1.0,1.0,1.0
20030,12728,0,0.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,45.0,30.0,45329000000.0,30.0,0 .. 5,41.20,NaN,NaN,NaN,0.0,4.0,3.0,3.0
23912,12729,0,0.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,45.0,30.0,45321000000.0,30.0,101 .. 150,46.46,NaN,NaN,NaN,RARE_VALUE,1.0,1.0,1.0


,id_client,id_contract,IP_flag,F1100,F1110,F1150,F1160,F1170,F1180,F1190,F1200,F1210,F1220,F1230,F1240,F1250,F1260,F1300,F1310,F1320,F1350,F1360,F1370,F1400,F1410,F1420,F1450,F1500,F1510,F1520,F1530,F1550,F1600,F1700,F2100,F2110,F2120,F2200,F2210,F2220,...,new3,new4,new5,new6,new7,new8,new9,new10,new11,new12,new13,new14,new15,new16,new17,new18,new19,new20,new21,new22,new23,new24,new25,new26,new27,new28,new_F1150,new_F1230,new_F1410,new_F1510,new_F1520,new_F2110,new_F2120,new_F2200,new_F2210,new_F2220,new_F2300,new_F2320,new_F2330,new_F2400
0,4,19084.0,0.0,2177000.0,NaN,2177000.0,NaN,NaN,NaN,NaN,6506000.0,2598000.0,NaN,1722000.0,NaN,2186000.0,NaN,6956000.0,10000.0,NaN,NaN,NaN,6946000.0,NaN,NaN,NaN,NaN,1727000.0,292000.0,1435000.0,NaN,NaN,8683000.0,8683000.0,-1145000.0,33927000.0,35072000.0,-1145000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1331000.0,-35000.0,NaN,-668000.0,-233000.0,-1906000.0,-149000.0,-1757000.0,NaN,NaN,-2144000.0,NaN,NaN,-2093000.0
1,6,24955.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.0,NaN,inf,0.0,1.0,1999602.0,0.0,1.0,1375253.2,0.000000,1.000000,129109.043478,0.000000,1.000000,145328.395122,2.0,NaN,2.0,1.666667,1.146272,4.456522,NaN,4.456522,4.456522,5.016373,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8,21717.0,0.0,3410000.0,NaN,2984000.0,NaN,NaN,NaN,426000.0,55010000.0,17909000.0,39000.0,21333000.0,NaN,15729000.0,NaN,15140000.0,500000.0,NaN,1542000.0,NaN,13098000.0,NaN,NaN,NaN,NaN,43280000.0,NaN,43280000.0,NaN,NaN,58420000.0,58420000.0,45028000.0,393065000.0,348037000.0,6894000.0,38134000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-436000.0,4862000.0,NaN,NaN,12624000.0,-85953000.0,-88777000.0,69000.0,2755000.0,NaN,436000.0,NaN,NaN,38000.0
3,18,19618.5,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7102000.0,3351000.0,NaN,1925000.0,NaN,1826000.0,NaN,3203000.0,10000.0,NaN,NaN,NaN,3193000.0,2925000.0,2925000.0,NaN,NaN,974000.0,NaN,974000.0,NaN,NaN,7102000.0,7102000.0,905000.0,10093000.0,9188000.0,905000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.030612,0.469388,220082.244444,0.031826,0.438861,178142.488095,NaN,NaN,NaN,NaN,NaN,6.091837,6.333333,5.695652,5.600000,4.532842,NaN,409000.0,-704000.0,NaN,662000.0,-1467000.0,-956000.0,-511000.0,NaN,NaN,-511000.0,NaN,NaN,NaN
4,19,18780.0,0.0,6276000.0,NaN,6276000.0,NaN,NaN,NaN,NaN,3651000.0,114000.0,NaN,1907000.0,NaN,1630000.0,NaN,2546000.0,10000.0,NaN,NaN,NaN,2536000.0,266000.0,266000.0,NaN,NaN,7115000.0,NaN,7115000.0,NaN,NaN,9927000.0,9927000.0,6470000.0,84375000.0,77905000.0,3117000.0,3353000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3894000.0,-5214000.0,-675000.0,NaN,-715000.0,21807000.0,19272000.0,685000.0,1850000.0,NaN,-160000.0,1000.0,NaN,-185000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4756,12725,20931.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,135000.0,NaN,3000.0,79000.0,NaN,28000.0,25000.0,17000.0,30000.0,NaN,NaN,NaN,-13000.0,NaN,NaN,NaN,NaN,118000.0,NaN,118000.0,NaN,NaN,135000.0,135000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,NaN,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4757,12726,21787.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [ ]:
last_train = pd.merge(konh, new_obj_df_tr, on='id_client', how='outer')

In [ ]:
last_test = pd.merge(konh_test, new_obj_df_te, on='id_client', how='outer')

In [ ]:
all_tune_df = pd.concat([last_train,last_test]).reset_index().drop(['index'],axis=1)

In [ ]:
all_tune_df#8922

,id_client,id_contract,IP_flag_x,F1100,F1110,F1150,F1160,F1170,F1180,F1190,F1200,F1210,F1220,F1230,F1240,F1250,F1260,F1300,F1310,F1320,F1350,F1360,F1370,F1400,F1410,F1420,F1450,F1500,F1510,F1520,F1530,F1550,F1600,F1700,F2100,F2110,F2120,F2200,F2210,F2220,...,new21,new22,new23,new24,new25,new26,new27,new28,new_F1150,new_F1230,new_F1410,new_F1510,new_F1520,new_F2110,new_F2120,new_F2200,new_F2210,new_F2220,new_F2300,new_F2320,new_F2330,new_F2400,IP_flag_y,EGRPOINCLUDED_y,OKFS_GROUP,OKOPF_GROUP,OKOGU_GROUP,OKATO_REGIONCODE_y,OKATO_FED_y,OKTMO_CODE_fixed,OKTMO_FED_y,WORKERSRANGE,OKVED_CODE_fixed,FLAG_DISQUALIFICATION_y,SEX_NAME,CITIZENSHIP_NAME,TELEPHONECOUNT_fixed,MANAGERCOUNTINCOUNTRY_fixed,MANAGERCOUNTINREGION_fixed,MANAGERINNCOUNT_fixed
0,1,5166.0,0.173076,8.203257e+08,NaN,5.973975e+08,NaN,NaN,NaN,NaN,3.432944e+08,NaN,NaN,NaN,NaN,3.863817e+07,NaN,2.439706e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.299378e+08,NaN,2.339637e+08,NaN,NaN,7.417830e+08,7.423574e+08,NaN,1.050932e+09,8.576320e+08,3.689082e+07,NaN,NaN,...,NaN,NaN,NaN,6.050000,2.0,5.045455,9.000000,4.686366,1.769131e+07,NaN,NaN,NaN,5.972605e+06,1.031203e+08,8.772140e+07,9.180000e+05,NaN,NaN,918000.0,NaN,NaN,918000.0,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,6 .. 10,43.11,NaN,NaN,NaN,1.0,1.0,1.0,1.0
1,2,2389.0,0.173076,8.203257e+08,NaN,5.973975e+08,NaN,1.088193e+09,NaN,NaN,3.432944e+08,6.303532e+07,NaN,2.065757e+08,NaN,3.863817e+07,NaN,2.439706e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.299378e+08,1.868574e+08,2.339637e+08,NaN,NaN,7.417830e+08,7.423574e+08,NaN,1.050932e+09,8.576320e+08,3.689082e+07,NaN,NaN,...,NaN,NaN,NaN,4.190476,2.0,3.153846,3.153846,4.306454,1.769131e+07,7.407014e+06,NaN,8525000.0,5.972605e+06,1.031203e+08,8.772140e+07,1.168000e+06,NaN,NaN,1368000.0,NaN,-200000.0,1095000.0,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,6 .. 10,31.09,NaN,NaN,NaN,2.0,1.0,1.0,1.0
2,3,17329.0,0.173076,8.203257e+08,NaN,5.973975e+08,NaN,NaN,NaN,NaN,3.432944e+08,6.303532e+07,1.083314e+07,2.065757e+08,NaN,3.863817e+07,1.196134e+07,2.439706e+08,1.068112e+08,NaN,NaN,NaN,NaN,5.198930e+08,7.444030e+08,NaN,NaN,3.299378e+08,NaN,2.339637e+08,NaN,NaN,7.417830e+08,7.423574e+08,3.379805e+08,1.050932e+09,8.576320e+08,3.689082e+07,4.616342e+08,9.613535e+07,...,NaN,NaN,NaN,2.000000,NaN,NaN,NaN,NaN,1.769131e+07,7.407014e+06,4.152455e+07,NaN,5.972605e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,0 .. 5,10.11,NaN,NaN,NaN,2.0,1.0,1.0,1.0
3,5,15086.0,0.173076,8.203257e+08,NaN,5.973975e+08,NaN,NaN,7.021363e+07,4.654814e+08,3.432944e+08,6.303532e+07,NaN,2.065757e+08,1.728200e+08,3.863817e+07,NaN,2.439706e+08,1.068112e+08,NaN,NaN,NaN,1.328897e+08,NaN,NaN,NaN,NaN,3.299378e+08,NaN,2.339637e+08,NaN,5.096975e+07,7.417830e+08,7.423574e+08,3.379805e+08,1.050932e+09,8.576320e+08,4.023600e+07,4.616342e+08,NaN,...,19.0,17.0,11.737408,6.058442,3.0,5.937716,5.023102,6.189865,1.769131e+07,7.407014e+06,NaN,NaN,5.972605e+06,1.031203e+08,8.772140e+07,2.166941e+07,6792000.0,NaN,23302000.0,256000.0,NaN,19022000.0,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,79701000001.0,40.0,251 .. 500,10.51,NaN,NaN,NaN,2.0,1.0,1.0,1.0
4,7,933.0,0.194078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,79.0,40.0,79701000001.0,40.0,NaN,47.52.7,NaN,мужской,Российская Федерация,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [ ]:
all_tune_df = all_tune_df.drop(['FLAG_DISQUALIFICATION_x','EGRPOINCLUDED_x','OKATO_REGIONCODE_x','OKATO_FED_x','OKTMO_FED_x'],axis=1)

In [ ]:
sorted(list(all_tune_df.new4.unique()))[-2]
l = [4,5,6,7,10,13,16,19,20,21,22,23,24,25,26,27,28]
def min_inf(x):
    if x==np.inf:
        return m**3
    return x
for x in l:
    m = sorted(list(all_tune_df['new'+str(x)].unique()))[-2]
    all_tune_df['new'+str(x)] = all_tune_df['new'+str(x)].apply(min_inf)


In [ ]:
new = 'new_F1150	new_F1230	new_F1410	new_F1510	new_F1520	new_F2110	new_F2120	new_F2200	new_F2210	new_F2220	new_F2300	new_F2320	new_F2330'
new = new.split()
new = new + ['new0','new1','new2','new3','new9','new12','new15','new18']

In [ ]:
for x in new:
    m = sorted(list(all_tune_df[x].unique()))[-2]
    all_tune_df[x] = all_tune_df[x].apply(min_inf)

In [ ]:
all_tune_df.describe()

,id_contract,IP_flag_x,F1100,F1110,F1150,F1160,F1170,F1180,F1190,F1200,F1210,F1220,F1230,F1240,F1250,F1260,F1300,F1310,F1320,F1350,F1360,F1370,F1400,F1410,F1420,F1450,F1500,F1510,F1520,F1530,F1550,F1600,F1700,F2100,F2110,F2120,F2200,F2210,F2220,F2300,...,new4,new5,new6,new7,new8,new9,new10,new11,new12,new13,new14,new15,new16,new17,new18,new19,new20,new21,new22,new23,new24,new25,new26,new27,new28,new_F1150,new_F1230,new_F1410,new_F1510,new_F1520,new_F2110,new_F2120,new_F2200,new_F2210,new_F2220,new_F2300,new_F2320,new_F2330,new_F2400,OKATO_REGIONCODE_y
count,13683.000000,13683.000000,5.610000e+03,8.110000e+02,5.315000e+03,1.790000e+02,1.028000e+03,1.423000e+03,9.680000e+02,9.383000e+03,7.865000e+03,2.171000e+03,8.989000e+03,1.817000e+03,9.171000e+03,2.873000e+03,9.389000e+03,5.586000e+03,4.400000e+01,7.640000e+02,8.920000e+02,5.521000e+03,3.111000e+03,1.960000e+03,1.375000e+03,5.640000e+02,9.126000e+03,3.948000e+03,9.028000e+03,5.960000e+02,6.540000e+02,9.410000e+03,9.409000e+03,5.604000e+03,9.142000e+03,8.889000e+03,9.175000e+03,2.883000e+03,2.323000e+03,9.200000e+03,...,1210.000000,1951.000000,2175.000000,3188.000000,3110.000000,2.456000e+03,4036.000000,3956.000000,3.326000e+03,5189.000000,5177.000000,4.879000e+03,5421.000000,5415.000000,5.176000e+03,3184.000000,1824.000000,2927.000000,2751.000000,2748.000000,5194.000000,3275.000000,5075.000000,4983.000000,4979.000000,4.568000e+03,7.758000e+03,1.255000e+03,2.701000e+03,7.774000e+03,7.836000e+03,7.597000e+03,7.882000e+03,2.387000e+03,1.748000e+03,7.915000e+03,1.480000e+03,1.854000e+03,7.917000e+03,12976.000000
mean,13355.363216,0.197537,1.412396e+09,7.338620e+07,9.821448e+08,6.054781e+07,1.474330e+09,8.579928e+07,6.072123e+08,4.535443e+08,8.764348e+07,1.544479e+07,2.711361e+08,2.384222e+08,6.672904e+07,1.862406e+07,5.630268e+08,3.219787e+08,-4.408110e+07,1.506131e+09,5.124268e+07,2.390273e+08,9.434506e+08,1.196561e+09,1.767468e+08,3.240034e+08,4.341711e+08,2.441830e+08,2.880453e+08,2.495982e+08,7.187815e+07,1.293915e+09,1.294736e+09,4.795331e+08,1.276705e+09,1.008055e+09,8.367592e+07,6.252659e+08,1.283470e+08,7.459662e+07,...,74.845194,51.880757,45.677422,2.810205,0.617694,2.442152e+06,2.249886,0.618910,2.005232e+06,0.303916,0.598294,1.885163e+06,0.177187,0.591309,1.692732e+06,9.055641,65.107643,23.024000,22.644073,240.633559,14.724868,76.924719,25.968331,19.286230,809.113448,7.296597e+07,2.906710e+07,2.366772e+08,3.281536e+07,1.823174e+07,1.325861e+08,1.124402e+08,5.472223e+06,1.128426e+08,1.642847e+07,-6.324845e+06,2.185065e+06,-1.177636e+06,-8.743966e+06,46.353961
std,7338.153942,0.396563,2.335014e+10,7.123029e+08,1.516030e+10,2.791174e+08,1.453201e+10,5.254806e+08,9.528816e+09,4.337571e+09,8.294997e+08,1.529111e+08,2.761346e+09,2.001061e+09,1.028739e+09,3.434938e+08,1.251072e+10,9.205797e+09,1.408190e+08,9.280902e+09,6.696838e+08,4.098107e+09,1.458239e+10,1.497680e+10,1.808590e+09,1.533942e+09,4.319518e+09,2.615102e+09,2.478468e+09,2.094002e+09,7.432335e+08,2.144236e+10,2.144365e+10,5.528855e+09,1.053177e+10,7.529282e+09,1.869891e+09,6.195642e+09,1.145533e+09,1.702799e+09,...,78.666615,93.907140,91.748461,16.506582,0.352511,2.369265e+07,14.712420,0.328314,8.108947e+06,5.288058,0.301812,1.096174e+07,3.798250,0.275649,1.178427e+07,32.860761,89.473407,66.863243,53.548384,650.675469,153.542174,109.739153,147.274730,95.264563,5475.701525,1.648856e+09,1.174745e+09,5.251975e+09,1.596507e+09,4.581930e+08,1.107200e+09,9.560856e+08,5.388847e+08,3.437756e+09,1.293163e+08,1.727470e+09,1.506244e+08,2.073128e+08,1.670505e+09,25.274766
min,2.000000,0.000000,-1.820000e+05,1.000000e+03,4.740000e+02,-1.175000e+06,1.000000e+03,-2.540000e+05,1.000000e+03,-2.950000e+05,-2.100000e+05,-3.990000e+05,-7.919000e+06,-5.200000e+04,-2.103100e+07,-3.701000e+06,-7.865320e+10,1.000000e+01,-8.538600e+08,-2.273630e+08,-5.030000e+05,-7.865321e+10,-4.140000e+06,-4.140000e+06,-7.150000e+05,-2.427000e+06,-6.193000e+06,-2.500000e+07,-1.903000e+06,-5.240000e+05,-8.498100e+07,-2.950000e+05,-2.950000e+05,-2.212800

In [ ]:
import scipy.stats as st

In [ ]:
data = all_tune_df['F1100']

In [ ]:
l,r = st.t.interval(alpha=0.99, df=len(data)-sum(np.isnan(data))-1, loc=np.mean(data), scale=np.std(data, ddof=1) / np.sqrt(np.size(data)))

In [ ]:
le_ri = []
def per_fil(x):
    if x<l:
        return l
    if x>r:
        return r
    return x
for x in all_tune_df.columns[2:162]:
    data = all_tune_df[x][(1+(-1)*np.isnan(all_tune_df[x])).apply(np.bool)]
    l,r = st.t.interval(alpha=0.99, df=len(data), loc=np.mean(data), scale=np.std(data, ddof=1) / np.sqrt(np.size(data)))
    le_ri.append(l)
    le_ri.append(r)
    all_tune_df[x] = all_tune_df[x].apply(per_fil)


In [ ]:
x = 'new_F2400'
data = all_tune_df[x][(1+(-1)*np.isnan(all_tune_df[x])).apply(np.bool)]
l,r = st.t.interval(alpha=0.99, df=len(data), loc=np.mean(data), scale=np.std(data, ddof=1) / np.sqrt(np.size(data)))
le_ri.append(l)
le_ri.append(r)
all_tune_df[x] = all_tune_df[x].apply(per_fil)

In [ ]:
all_tune_df = all_tune_df.drop(['IP_flag_x'],axis=1)

In [ ]:
#mmean
#sstd
def std_scal(e):
    return (e-mmean)/sstd

In [ ]:
for x in all_tune_df.columns[2:162]:
    data = all_tune_df[x][(1+(-1)*np.isnan(all_tune_df[x])).apply(np.bool)]
    mmean = np.mean(data)
    sstd = np.std(data)
    all_tune_df[x] = all_tune_df[x].apply(std_scal)

In [ ]:
all_tune_df

,id_client,id_contract,F1100,F1110,F1150,F1160,F1170,F1180,F1190,F1200,F1210,F1220,F1230,F1240,F1250,F1260,F1300,F1310,F1320,F1350,F1360,F1370,F1400,F1410,F1420,F1450,F1500,F1510,F1520,F1530,F1550,F1600,F1700,F2100,F2110,F2120,F2200,F2210,F2220,F2300,...,new21,new22,new23,new24,new25,new26,new27,new28,new_F1150,new_F1230,new_F1410,new_F1510,new_F1520,new_F2110,new_F2120,new_F2200,new_F2210,new_F2220,new_F2300,new_F2320,new_F2330,new_F2400,IP_flag_y,EGRPOINCLUDED_y,OKFS_GROUP,OKOPF_GROUP,OKOGU_GROUP,OKATO_REGIONCODE_y,OKATO_FED_y,OKTMO_CODE_fixed,OKTMO_FED_y,WORKERSRANGE,OKVED_CODE_fixed,FLAG_DISQUALIFICATION_y,SEX_NAME,CITIZENSHIP_NAME,TELEPHONECOUNT_fixed,MANAGERCOUNTINCOUNTRY_fixed,MANAGERCOUNTINREGION_fixed,MANAGERINNCOUNT_fixed
0,1,5166.0,-0.237203,NaN,-0.247628,NaN,NaN,NaN,NaN,-0.327317,NaN,NaN,NaN,NaN,-0.265977,NaN,-0.250692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.311619,NaN,-0.33284,NaN,NaN,-0.27275,-0.272697,NaN,-0.313297,-0.327784,-0.277209,NaN,NaN,-0.271915,...,NaN,NaN,NaN,-0.307364,-0.665809,-0.219946,-0.257933,-0.14951,-0.286706,NaN,NaN,NaN,-0.511012,-0.368548,-0.386088,-0.064579,NaN,NaN,0.028137,NaN,NaN,0.052724,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,6 .. 10,43.11,NaN,NaN,NaN,1.0,1.0,1.0,1.0
1,2,2389.0,-0.237203,NaN,-0.247628,NaN,-0.250965,NaN,NaN,-0.327317,-0.348328,NaN,-0.319713,NaN,-0.265977,NaN,-0.250692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.311619,-0.269537,-0.33284,NaN,NaN,-0.27275,-0.272697,NaN,-0.313297,-0.327784,-0.277209,NaN,NaN,-0.271915,...,NaN,NaN,NaN,-0.307364,-0.665809,-0.219946,-0.257933,-0.14951,-0.286706,-0.126129,NaN,0.118212,-0.511012,-0.368548,-0.386088,-0.032348,NaN,NaN,0.056254,NaN,0.04246,0.064745,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,6 .. 10,31.09,NaN,NaN,NaN,2.0,1.0,1.0,1.0
2,3,17329.0,-0.237203,NaN,-0.247628,NaN,NaN,NaN,NaN,-0.327317,-0.348328,-0.387576,-0.319713,NaN,-0.265977,-0.34809,-0.250692,-0.245664,NaN,NaN,NaN,NaN,-0.233301,-0.243715,NaN,NaN,-0.311619,NaN,-0.33284,NaN,NaN,-0.27275,-0.272697,-0.279925,-0.313297,-0.327784,-0.277209,-0.298098,-0.396543,-0.271915,...,NaN,NaN,NaN,-0.307364,NaN,NaN,NaN,NaN,-0.286706,-0.571415,-0.196372,NaN,-0.511012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,0 .. 5,10.11,NaN,NaN,NaN,2.0,1.0,1.0,1.0
3,5,15086.0,-0.237203,NaN,-0.247628,NaN,NaN,-0.390905,-0.323928,-0.327317,-0.348328,NaN,-0.319713,-0.326928,-0.265977,NaN,-0.250692,-0.194176,NaN,NaN,NaN,-0.254640,NaN,NaN,NaN,NaN,-0.311619,NaN,-0.33284,NaN,-0.35222,-0.27275,-0.272697,-0.279925,-0.313297,-0.327784,0.031521,-0.298098,NaN,-0.271915,...,-0.311058,-0.371822,-0.366503,-0.307364,-0.665809,-0.219946,-0.257933,-0.14951,-0.286706,-0.347558,NaN,NaN,-0.511012,-0.368548,-0.386088,2.538741,-0.188024,NaN,1.426763,0.013589,NaN,1.282236,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,79701000001.0,40.0,251 .. 500,10.51,NaN,NaN,NaN,2.0,1.0,1.0,1.0
4,7,933.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,79.0,40.0,79701000001.0,40.0,NaN,47.52.7,NaN,мужской,Российская Федерация,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13678,12725,20931.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.327317,NaN,-0.38757

In [ ]:
!pip install lightautoml

     |████████████████████████████████| 293 kB 12.5 MB/s 
     |████████████████████████████████| 308 kB 57.2 MB/s 
     |████████████████████████████████| 76.3 MB 39 kB/s 
     |████████████████████████████████| 424 kB 58.5 MB/s 
     |████████████████████████████████| 15.7 MB 60 kB/s 
     |████████████████████████████████| 1.2 MB 40.7 MB/s 
     |████████████████████████████████| 204 kB 55.4 MB/s 
     |████████████████████████████████| 804.1 MB 2.3 kB/s 
     |████████████████████████████████| 2.8 MB 72.6 MB/s 
     |████████████████████████████████| 80 kB 8.8 MB/s 
     |████████████████████████████████| 209 kB 77.8 MB/s 
     |████████████████████████████████| 75 kB 4.2 MB/s 
     |████████████████████████████████| 148 kB 54.0 MB/s 
     |████████████████████████████████| 112 kB 57.4 MB/s 
     |████████████████████████████████| 49 kB 5.7 MB/s 
     |████████████████████████████████| 575 kB 65.9 MB/s 
     |████████████████████████████████| 23.3 MB 1.3 MB/s 
     |███████████████

In [ ]:
# отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

# будем отображать графики прямо в jupyter'e
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 

#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 8, 5
import pandas as pd
# Standard python libraries
import logging
import os
import time
import requests
logging.basicConfig(format='[%(asctime)s] (%(levelname)s): %(message)s', level=logging.INFO)
import lightautoml

# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import torch

# Imports from our package
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task

import multiprocessing as mp

# set option of max 100 displayed rows
pd.set_option('display.max_rows', 200)

In [ ]:
N_THREADS = mp.cpu_count() # threads cnt for lgbm and linear models
N_FOLDS = 10 # folds cnt for AutoML - 10 is better
RANDOM_STATE = 42 # fixed random state for various reasons
TIMEOUT = 3600 # Time in seconds for automl run
TARGET_NAME = 'TARGET' # Target column name

In [ ]:
%%time

roles = {'target': TARGET_NAME,
         'drop': ['id_contract','id_client']
         }

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 9.78 µs


In [ ]:
%%time

task = Task('binary', loss='logloss', metric='auc')

CPU times: user 4.59 ms, sys: 5 µs, total: 4.6 ms
Wall time: 11.6 ms


In [ ]:
import random
def n05(x):
    if x>0.5:
        return 1
    if x==0.5:
        return random.choice([0, 1])
    return 0


In [ ]:
train_tune_df = all_tune_df.iloc[:8922]

In [ ]:
konh1 = pd.read_csv('/content/konh.csv')

In [ ]:
labels_tune = konh1.TARGET

In [ ]:
train_tune_df['TARGET'] = labels_tune.apply(n05)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
train_tune_df

,id_client,id_contract,F1100,F1110,F1150,F1160,F1170,F1180,F1190,F1200,F1210,F1220,F1230,F1240,F1250,F1260,F1300,F1310,F1320,F1350,F1360,F1370,F1400,F1410,F1420,F1450,F1500,F1510,F1520,F1530,F1550,F1600,F1700,F2100,F2110,F2120,F2200,F2210,F2220,F2300,...,new22,new23,new24,new25,new26,new27,new28,new_F1150,new_F1230,new_F1410,new_F1510,new_F1520,new_F2110,new_F2120,new_F2200,new_F2210,new_F2220,new_F2300,new_F2320,new_F2330,new_F2400,IP_flag_y,EGRPOINCLUDED_y,OKFS_GROUP,OKOPF_GROUP,OKOGU_GROUP,OKATO_REGIONCODE_y,OKATO_FED_y,OKTMO_CODE_fixed,OKTMO_FED_y,WORKERSRANGE,OKVED_CODE_fixed,FLAG_DISQUALIFICATION_y,SEX_NAME,CITIZENSHIP_NAME,TELEPHONECOUNT_fixed,MANAGERCOUNTINCOUNTRY_fixed,MANAGERCOUNTINREGION_fixed,MANAGERINNCOUNT_fixed,TARGET
0,1,5166.0,-0.237203,NaN,-0.247628,NaN,NaN,NaN,NaN,-0.327317,NaN,NaN,NaN,NaN,-0.265977,NaN,-0.250692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.311619,NaN,-0.33284,NaN,NaN,-0.27275,-0.272697,NaN,-0.313297,-0.327784,-0.277209,NaN,NaN,-0.271915,...,NaN,NaN,-0.307364,-0.665809,-0.219946,-0.257933,-0.14951,-0.286706,NaN,NaN,NaN,-0.511012,-0.368548,-0.386088,-0.064579,NaN,NaN,0.028137,NaN,NaN,0.052724,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,6 .. 10,43.11,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1
1,2,2389.0,-0.237203,NaN,-0.247628,NaN,-0.250965,NaN,NaN,-0.327317,-0.348328,NaN,-0.319713,NaN,-0.265977,NaN,-0.250692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.311619,-0.269537,-0.33284,NaN,NaN,-0.27275,-0.272697,NaN,-0.313297,-0.327784,-0.277209,NaN,NaN,-0.271915,...,NaN,NaN,-0.307364,-0.665809,-0.219946,-0.257933,-0.14951,-0.286706,-0.126129,NaN,0.118212,-0.511012,-0.368548,-0.386088,-0.032348,NaN,NaN,0.056254,NaN,0.04246,0.064745,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,6 .. 10,31.09,NaN,NaN,NaN,2.0,1.0,1.0,1.0,0
2,3,17329.0,-0.237203,NaN,-0.247628,NaN,NaN,NaN,NaN,-0.327317,-0.348328,-0.387576,-0.319713,NaN,-0.265977,-0.34809,-0.250692,-0.245664,NaN,NaN,NaN,NaN,-0.233301,-0.243715,NaN,NaN,-0.311619,NaN,-0.33284,NaN,NaN,-0.27275,-0.272697,-0.279925,-0.313297,-0.327784,-0.277209,-0.298098,-0.396543,-0.271915,...,NaN,NaN,-0.307364,NaN,NaN,NaN,NaN,-0.286706,-0.571415,-0.196372,NaN,-0.511012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,0 .. 5,10.11,NaN,NaN,NaN,2.0,1.0,1.0,1.0,1
3,5,15086.0,-0.237203,NaN,-0.247628,NaN,NaN,-0.390905,-0.323928,-0.327317,-0.348328,NaN,-0.319713,-0.326928,-0.265977,NaN,-0.250692,-0.194176,NaN,NaN,NaN,-0.25464,NaN,NaN,NaN,NaN,-0.311619,NaN,-0.33284,NaN,-0.35222,-0.27275,-0.272697,-0.279925,-0.313297,-0.327784,0.031521,-0.298098,NaN,-0.271915,...,-0.371822,-0.366503,-0.307364,-0.665809,-0.219946,-0.257933,-0.14951,-0.286706,-0.347558,NaN,NaN,-0.511012,-0.368548,-0.386088,2.538741,-0.188024,NaN,1.426763,0.013589,NaN,1.282236,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,79701000001.0,40.0,251 .. 500,10.51,NaN,NaN,NaN,2.0,1.0,1.0,1.0,0
4,7,933.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,79.0,40.0,79701000001.0,40.0,NaN,47.52.7,NaN,мужской,Российская Федерация,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8917,12731,10258.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [ ]:
#### %%time 
automl = TabularUtilizedAutoML(task = task, 
                       timeout = 2400,
                       cpu_limit = N_THREADS,
                       #general_params = {'use_algos': [['linear_l2', 'lgb', 'lgb_tuned']]},
                       reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE},
                      )
oof_pred = automl.fit_predict(train_tune_df[top30], roles = roles)
logging.info('oof_pred:\n{}\nShape = {}'.format(oof_pred, oof_pred.shape))

In [ ]:
roc_auc_score(train_tune_df.TARGET,oof_pred.data)

0.8311992654122304

In [ ]:
automl.get_feature_scores()

,Feature,Importance
0,OKVED_CODE_fixed,27979.557280
1,WORKERSRANGE,21659.944173
2,OKTMO_CODE_fixed,8905.173985
3,OKATO_REGIONCODE_y,6768.158902
4,new16,6140.499060
5,new_F2300,5137.535119
6,new_F2200,4626.629576
7,new_F2400,4162.476371
8,OKFS_GROUP,4137.587962
9,TELEPHONECOUNT_fixed,3587.554306


In [ ]:
top30 = feature_score[:30]

In [ ]:
top30

['id_client',
 'id_contract',
 'TARGET',
 'OKVED_CODE_fixed',
 'WORKERSRANGE',
 'OKTMO_CODE_fixed',
 'new16',
 'OKATO_REGIONCODE_y',
 'new_F2200',
 'TELEPHONECOUNT_fixed',
 'new_F2300',
 'new_F2400',
 'OKFS_GROUP',
 'OKATO_FED_y',
 'F2410',
 'new1',
 'AGE',
 'SIGNEDNUMBER_95_EVER',
 'new15',
 'new_F1520',
 'new_F1150',
 'new25',
 'MANAGERCOUNTINCOUNTRY_fixed',
 'new_F2110',
 'new_F2210',
 'F2340',
 'new11',
 'new_F2330',
 'new2',
 'F1210',
 'F2200_LAG1',
 'new8',
 'new_F1230']

In [ ]:
top30 = list(top30.Feature.array)

AttributeError: ignored

In [ ]:
top30 = ['id_client','id_contract','TARGET']+top30

In [ ]:
top30 = ['id_client','id_contract','TARGET']

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(train_tune_df.TARGET,oof_pred.data)

0.8335662485751009

In [ ]:
feature_score = automl.get_feature_scores()

In [ ]:
test_pred = automl.predict(all_tune_df.loc[8922:,top30[3:]])

In [ ]:
test_pred

array([[0.06262112],
       [0.01364326],
       [0.0101991 ],
       ...,
       [0.8571751 ],
       [0.03476048],
       [0.48198575]], dtype=float32)

In [ ]:
otpr_df = pd.DataFrame()

In [ ]:
otpr_df['id_client'] = all_tune_df.iloc[8922:,:]['id_client']

In [ ]:
otpr_df['TARGET'] = test_pred.data

In [ ]:
tar = []

In [ ]:
id = list(train_tune_df['id_client'].array)

In [ ]:
dec = dict(zip(id,labels_tune))

In [ ]:
id_test = list(otpr_df['id_client'].array)

In [ ]:
test_pred = test_pred.data.squeeze()

In [ ]:
test_pred = list(test_pred)

In [ ]:
for i in range(len(id_test)):
    if id_test[i] in id:
        tar.append(dec[id_test[i]])
    else:
        tar.append(test_pred[i])

In [ ]:
otpr_df['TARGET'] = tar

In [ ]:
dec_test = dict(zip(list(otpr_df['id_client'].array),tar))

In [ ]:
last_test_df = all_df.loc[17891:,['id_client','id_contract']]

In [ ]:
last_tar = []
last_id = list(last_test_df['id_client'].array)

In [ ]:
for x in last_id:
    last_tar.append(dec_test[x])

In [ ]:
last_test_df['TARGET'] = last_tar

In [ ]:
last_test_df

,id_client,id_contract,TARGET
17891,3620,17892,0.000000
17892,4101,17893,0.000000
17893,9589,17894,0.131144
17894,11546,17895,0.271729
17895,12558,17896,1.000000
...,...,...,...
25216,8128,25217,0.449205
25217,1132,25218,0.000000
25218,4932,25219,0.364439
25219,537,25220,0.549814


In [ ]:
last_test_df.drop(['id_client'],axis=1).to_csv('trying_scaled.csv', index=False,sep=';')

In [ ]:
a1 = pd.read_csv('/content/trying_scaled.csv',sep=';')
a2 = pd.read_csv('/content/trying_ouh3aaa.csv',sep=';')

In [ ]:
t1 = a1.TARGET
t2 = a2.TARGET

In [ ]:
a1.TARGET = (t1+t2)/2

In [ ]:
a1.to_csv('trying_scaled+blend.csv', index=False,sep=';')

In [ ]:
test_pred.data.squeeze()

array([0.06262112, 0.01364326, 0.0101991 , ..., 0.8571751 , 0.03476048,
       0.48198575], dtype=float32)

In [ ]:
a3.to_csv('/content/trying_scaled+blend3.csv',index=False,sep=';')

In [ ]:
a4 = pd.read_csv('/content/trying_scaled+blend.csv',sep=';')

In [ ]:
last_test_df.TARGET

NameError: ignored

In [ ]:
a3 = pd.read_csv('/content/trying_ouh3aaa.csv',sep=';')

In [ ]:
a3['TARGET'] = (a3.TARGET.array+last_test_df.TARGET.array)/2

In [ ]:
(a3.TARGET.array+last_test_df.TARGET.array)/2

<PandasArray>
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
 ...
 nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
Length: 7330, dtype: float64

In [ ]:
automl.get_feature_scores()

,Feature,Importance
0,OKVED_CODE_fixed,27979.557280
1,WORKERSRANGE,21659.944173
2,OKTMO_CODE_fixed,8905.173985
3,OKATO_REGIONCODE_y,6768.158902
4,new16,6140.499060
5,new_F2300,5137.535119
6,new_F2200,4626.629576
7,new_F2400,4162.476371
8,OKFS_GROUP,4137.587962
9,TELEPHONECOUNT_fixed,3587.554306


In [ ]:
sum(all_df.OKVED_CODE_fixed.value_counts())

23739

In [ ]:
all_df.OKVED_CODE_fixed.value_counts()

46.46         2692
RARE_VALUE    1679
41.20          993
46.46.1        779
46.90          610
              ... 
47.91           10
71.20.6         10
46.42.11        10
86.23           10
35.30.11        10
Name: OKVED_CODE_fixed, Length: 299, dtype: int64

,id_contract,id_client,IP_flag,F1100,F1110,F1150,F1160,F1170,F1180,F1190,F1200,F1210,F1220,F1230,F1240,F1250,F1260,F1300,F1310,F1320,F1350,F1360,F1370,F1400,F1410,F1420,F1450,F1500,F1510,F1520,F1530,F1550,F1600,F1700,F2100,F2110,F2120,F2200,F2210,F2220,...,new3,new4,new5,new6,new7,new8,new9,new10,new11,new12,new13,new14,new15,new16,new17,new18,new19,new20,new21,new22,new23,new24,new25,new26,new27,new28,new_F1150,new_F1230,new_F1410,new_F1510,new_F1520,new_F2110,new_F2120,new_F2200,new_F2210,new_F2220,new_F2300,new_F2320,new_F2330,new_F2400
0,1,1847,0,1.298961e+09,2154000.0,1.125573e+09,NaN,150010000.0,NaN,21224000.0,1.048287e+10,105901000.0,127510000.0,9.477348e+09,440671000.0,3.312980e+08,138000.0,1.960896e+09,1.000700e+07,-46634000.0,599497000.0,500000.0,1.028878e+09,340135000.0,NaN,340135000.0,NaN,9.480796e+09,2.499973e+09,6.907960e+09,NaN,NaN,1.178183e+10,1.178183e+10,2.370882e+09,5.294104e+10,5.057016e+10,631752000.0,1.083075e+09,656055000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93873000.0,298288000.0,NaN,387793000.0,-518544000.0,3.383797e+09,4.083358e+09,75244000.0,-341715000.0,-433090000.0,-151767000.0,-30707000.0,-16692000.0,-37171000.0
1,2,4650,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,4770,0,7.337400e+07,NaN,7.337400e+07,NaN,NaN,NaN,NaN,2.161000e+07,2378000.0,NaN,1.764200e+07,1000000.0,5.530000e+05,37000.0,9.073600e+07,1.000000e+05,NaN,6780000.0,NaN,8.385600e+07,NaN,NaN,NaN,NaN,4.248000e+06,NaN,4.171000e+06,77000.0,NaN,9.498400e+07,9.498400e+07,9.315000e+06,5.218400e+07,4.286900e+07,2569000.0,NaN,6746000.0,...,22631.974359,1.333333,39.0,3.5,0.0,1.0,1174998.0,0.000000,1.000000,1369108.5,0.0,1.00,1.079478e+05,0.000000,1.000000,1.092365e+05,2.0,NaN,2.0,2.0,2.330401,4.447368,NaN,4.447368,4.307692,4.359121,2700000.0,-615000.0,NaN,NaN,407000.0,8.440000e+05,-1.300000e+05,-680000.0,NaN,1654000.0,-75000.0,NaN,NaN,-25000.0
3,4,12237,0,1.937488e+09,122828000.0,6.103280e+08,NaN,809426000.0,356828000.0,38078000.0,5.786965e+09,12001000.0,71000.0,1.232486e+09,78603000.0,4.459193e+09,4611000.0,2.685905e+09,1.516514e+09,NaN,NaN,65834000.0,1.103557e+09,627253000.0,NaN,689000.0,626564000.0,4.411295e+09,3.930000e+05,3.092581e+09,NaN,NaN,7.724453e+09,7.724453e+09,1.694054e+10,4.028466e+10,2.334412e+10,931118000.0,1.600942e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17348000.0,-166335000.0,NaN,115000.0,-894886000.0,1.553888e+09,1.636525e+09,-101602000.0,18965000.0,NaN,-35934000.0,28228000.0,3617000.0,-99194000.0
4,5,9988,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25216,25217,8128,0,1.395000e+06,NaN,1.395000e+06,NaN,NaN,NaN,NaN,5.150000e+05,163000.0,NaN,3.180000e+05,NaN,3.400000e+04,NaN,4.400000e+04,NaN,NaN,NaN,NaN,NaN,726000.0,NaN,NaN,726000.0,1.140000e+06,3.000000e+05,8.400000e+05,NaN,NaN,1.910000e+06,1.910000e+06,NaN,1.518000e+06,1.424000e+06,94000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.00,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,308000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [ ]:
all_tune_df.groupby('OKTMO_CODE_fixed').aggregate('mean').reset_index()

,OKTMO_CODE_fixed,id_contract,F1100,F1110,F1150,F1160,F1170,F1180,F1190,F1200,F1210,F1220,F1230,F1240,F1250,F1260,F1300,F1310,F1320,F1350,F1360,F1370,F1400,F1410,F1420,F1450,F1500,F1510,F1520,F1530,F1550,F1600,F1700,F2100,F2110,F2120,F2200,F2210,F2220,F2300,...,new4,new5,new6,new7,new8,new9,new10,new11,new12,new13,new14,new15,new16,new17,new18,new19,new20,new21,new22,new23,new24,new25,new26,new27,new28,new_F1150,new_F1230,new_F1410,new_F1510,new_F1520,new_F2110,new_F2120,new_F2200,new_F2210,new_F2220,new_F2300,new_F2320,new_F2330,new_F2400,OKATO_REGIONCODE_y
0,10701000001.0,11537.684211,-0.237203,NaN,-0.247628,NaN,NaN,NaN,NaN,-0.327317,-0.348328,NaN,-0.319713,-0.326928,-0.265977,-0.348090,-0.250692,-0.245664,NaN,NaN,NaN,-0.351804,-0.233301,-0.243715,-0.275226,NaN,-0.311619,-0.269537,-0.332840,NaN,-0.349573,-0.272750,-0.272697,-0.279925,-0.313297,-0.327784,-0.277209,-0.298098,NaN,-0.271915,...,-0.248302,0.393186,-0.165353,-0.176692,0.240851,-0.159088,-0.036938,-0.106328,-0.125573,0.010050,0.313443,0.109654,-0.023540,0.576423,0.014598,-0.032483,0.219610,-0.311058,-0.064076,-0.056553,-0.086847,0.298200,-0.144484,0.065110,-0.149510,-0.286706,0.050162,-0.170039,-0.168078,-0.205364,-0.351105,-0.308882,-0.078149,-0.284468,NaN,0.012897,NaN,0.139390,0.029080,10.000000
1,10730000001.0,10812.948980,-0.237203,NaN,-0.247628,NaN,NaN,NaN,NaN,-0.327317,-0.348328,-0.387576,-0.319713,NaN,0.586298,-0.348090,-0.250692,-0.245664,NaN,NaN,NaN,0.351911,NaN,NaN,NaN,NaN,-0.311619,-0.269537,-0.332840,NaN,NaN,-0.272750,-0.272697,-0.279925,-0.313297,-0.327784,-0.277209,-0.298098,-0.396543,-0.271915,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.239127,1.043343,-0.445877,-0.367981,1.050709,-0.132111,NaN,NaN,NaN,NaN,NaN,-0.307364,NaN,-0.219946,-0.257933,-0.149510,-0.286706,0.151977,NaN,0.339489,-0.041844,-0.281355,-0.307399,0.009629,-0.127056,-0.437026,0.126373,NaN,NaN,0.176388,10.000000
2,11701000001.0,14949.181818,-0.152456,-0.352062,-0.106519,NaN,-0.250965,0.700889,-0.301586,0.071797,-0.094367,-0.387576,0.099643,-0.326928,-0.097878,-0.348090,-0.135571,0.254026,NaN,-0.243580,-0.334664,-0.156177,0.374029,0.250056,-0.275226,0.207385,0.058004,0.006338,0.067199,-0.240582,-0.344620,0.184790,0.184711,0.095091,0.041435,0.090441,-0.008277,0.137650,-0.283503,-0.019742,...,0.588127,-0.521637,0.334295,-0.176692,-0.110710,-0.111803,-0.164569,-0.426654,-0.224719,-0.088306,-0.238113,-0.445877,0.042506,-0.346034,-0.364902,-0.081000,-0.390083,0.014399,0.141087,0.150081,-0.205019,-0.232005,-0.219946,-0.257933,-0.149510,0.235199,-0.168583,0.818370,-0.582065,-0.053122,-0.102928,-0.080152,0.051014,-0.125894,-0.437026,0.056984,0.426303,-0.862571,0.068197,11.000000
3,11851000001.0,17891.850000,0.242078,NaN,0.858194,NaN,NaN,-0.390905,-0.288069,0.387846,0.798175,-0.387576,-0.003648,NaN,0.418647,0.340022,0.718941,1.094263,NaN,0.811352,0.020757,-0.351804,-0.233301,NaN,-0.275226,NaN,0.429052,0.020409,-0.332840,1.799185,NaN,0.595420,0.595581,-0.279925,0.227792,0.560944,-0.277209,NaN,0.724074,-0.271915,...,0.086270,-0.521637,0.863333,NaN,NaN,NaN,1.749896,0.968317,-0.457604,-0.239127,0.759871,0.511272,0.883199,0.546780,0.984641,NaN,NaN,NaN,NaN,NaN,-0.307364,NaN,-0.219946,-0.257933,-0.149510,0.484902,0.025568,NaN,1.049694,-0.277486,-0.368548,0.404690,-0.152813,NaN,-0.437026,0.013028,0.349720,-0.174102,0.110804,11.000000
4,12701000001.0,13547.480247,0.151149,NaN,0.309196,NaN,-0.250965,1.620140,-0.254721,0.139094,-0.274074,1.214840,0.158900,-0.326928,-0.055224,NaN,0.083832,-0.245156,0.411475,0.371483,-0.403861,-0.070922,-0.233301,-0.243715,0.117830,NaN,0.332443,0.395340,0.275915,-0.417643,-0.165584,0.195288,0.195264,0.017859,0.094414,0.156977,-0.080230,0.355524,0.275828,-0.198325,...,-0.917445,-0.115049,0.179137,-0.176692,0.636943,0.062788,-0.164569,-0.039162,-0.058373,-0.239127,-0.058267,-0.445877,-0.259465,-0.173785,-0.317408,-0.347843,0.372033,-0.311058,-0.371822,-0.366503,0.558732,0.057198,0.709654,0.568864,0.473467,-0.207006,-0.231999,-0.203005,-0.348179,-0.017045,-0.085183,-0.098532,-0

In [ ]:
d10 = [1,2,3,4,5,6,7,8,9]
def del10(x):
    if np.isnan(x):
        return x
    for i in range(1,10):
        if x<=d10[i]:
            return i

545

In [ ]:
for x in num_columns:
    d10 =[]
    l = min(train_df[x])
    r = max(train_df[x])
    razn = r-l
    for i in range(10):
        d10.append(l+razn*0.1*i)
    train_df[x] = train_df[x].apply(del10)
    test_df[x] = test_df[x].apply(del10)

8.340066415257752e-09

In [ ]:
all_tune_df2 = all_tune_df.copy()

In [ ]:
all_tune_df2

,id_client,id_contract,F1100,F1110,F1150,F1160,F1170,F1180,F1190,F1200,F1210,F1220,F1230,F1240,F1250,F1260,F1300,F1310,F1320,F1350,F1360,F1370,F1400,F1410,F1420,F1450,F1500,F1510,F1520,F1530,F1550,F1600,F1700,F2100,F2110,F2120,F2200,F2210,F2220,F2300,...,new22,new23,new24,new25,new26,new27,new28,new_F1150,new_F1230,new_F1410,new_F1510,new_F1520,new_F2110,new_F2120,new_F2200,new_F2210,new_F2220,new_F2300,new_F2320,new_F2330,new_F2400,IP_flag_y,EGRPOINCLUDED_y,OKFS_GROUP,OKOPF_GROUP,OKOGU_GROUP,OKATO_REGIONCODE_y,OKATO_FED_y,OKTMO_CODE_fixed,OKTMO_FED_y,WORKERSRANGE,OKVED_CODE_fixed,FLAG_DISQUALIFICATION_y,SEX_NAME,CITIZENSHIP_NAME,TELEPHONECOUNT_fixed,MANAGERCOUNTINCOUNTRY_fixed,MANAGERCOUNTINREGION_fixed,MANAGERINNCOUNT_fixed,new_double
0,1,5166.0,0.208284,NaN,0.149574,NaN,NaN,NaN,NaN,-0.124295,NaN,NaN,NaN,NaN,-0.064260,NaN,-0.162661,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.026461,NaN,0.028046,NaN,NaN,-0.048987,-0.049455,NaN,0.104026,0.140323,0.024362,NaN,NaN,-0.162694,...,NaN,NaN,-0.307364,-0.665809,-0.219946,-0.257933,-0.14951,-0.066721,NaN,NaN,NaN,-0.442729,-0.249409,-0.205258,-0.088985,NaN,NaN,0.028137,NaN,NaN,0.052724,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,6 .. 10,43.11,NaN,NaN,NaN,1.0,1.0,1.0,1.0,RARE_VALUE6 .. 10
1,2,2389.0,0.208284,NaN,0.149574,NaN,0.393885,NaN,NaN,-0.124295,-0.362389,NaN,-0.010348,NaN,-0.064260,NaN,-0.162661,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.026461,0.316405,0.028046,NaN,NaN,-0.048987,-0.049455,NaN,0.104026,0.140323,0.024362,NaN,NaN,-0.162694,...,NaN,NaN,-0.307364,-0.665809,-0.219946,-0.257933,-0.14951,-0.066721,-0.196132,NaN,0.06081,-0.442729,-0.249409,-0.205258,-0.026097,NaN,NaN,0.056254,NaN,0.04246,0.064745,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,6 .. 10,31.09,NaN,NaN,NaN,2.0,1.0,1.0,1.0,RARE_VALUE6 .. 10
2,3,17329.0,0.208284,NaN,0.149574,NaN,NaN,NaN,NaN,-0.124295,-0.362389,0.010485,-0.010348,NaN,-0.064260,0.390438,-0.162661,-0.023845,NaN,NaN,NaN,NaN,0.257969,0.316353,NaN,NaN,-0.026461,NaN,0.028046,NaN,NaN,-0.048987,-0.049455,0.085165,0.104026,0.140323,0.024362,0.177759,-0.028709,-0.162694,...,NaN,NaN,-0.307364,NaN,NaN,NaN,NaN,-0.066721,-0.196132,0.026536,NaN,-0.442729,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,0 .. 5,10.11,NaN,NaN,NaN,2.0,1.0,1.0,1.0,RARE_VALUE0 .. 5
3,5,15086.0,0.208284,NaN,0.149574,NaN,NaN,-0.097152,0.602864,-0.124295,-0.362389,NaN,-0.010348,0.073625,-0.064260,NaN,-0.162661,-0.023845,NaN,NaN,NaN,-0.116428,NaN,NaN,NaN,NaN,-0.026461,NaN,0.028046,NaN,0.241248,-0.048987,-0.049455,0.085165,0.104026,0.140323,0.267861,0.177759,NaN,-0.162694,...,-0.371822,-0.366503,-0.307364,-0.665809,-0.219946,-0.257933,-0.14951,-0.066721,-0.196132,NaN,NaN,-0.442729,-0.249409,-0.205258,1.965846,-0.188024,NaN,1.426763,0.013589,NaN,1.282236,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,79701000001.0,40.0,251 .. 500,10.51,NaN,NaN,NaN,2.0,1.0,1.0,1.0,79701000001.0251 .. 500
4,7,933.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,79.0,40.0,79701000001.0,40.0,NaN,47.52.7,NaN,мужской,Российская Федерация,NaN,NaN,NaN,NaN,79701000001.0nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13678,12725,20931.0,NaN,N

In [ ]:
def mean_nan(a):
    s = 0
    n = 0
    for x in a:
        if (1-np.isnan(x)):
            s+=x
            n+=1
    if n==0:
        return np.nan
    return s/n

In [ ]:
mean_nan([np.nan,np.nan,np.nan])

nan

In [ ]:
all_tune_df[['F1150']].fillna(all_tune_df.groupby('new_double').transform(mean_nan))

,F1150
0,0.149574
1,0.149574
2,0.149574
3,0.149574
4,NaN
...,...
13678,-0.230667
13679,NaN
13680,NaN
13681,NaN


In [ ]:
top30_test = ['id_client',
 'id_contract',
 'OKVED_CODE_fixed',
 'WORKERSRANGE',
 'OKTMO_CODE_fixed',
 'new16',
 'OKATO_REGIONCODE_y',
 'new_F2200',
 'TELEPHONECOUNT_fixed',
 'new_F2300',
 'new_F2400',
 'OKFS_GROUP',
 'OKATO_FED_y',
 'F2410',
 'new1',
 'AGE',
 'SIGNEDNUMBER_95_EVER',
 'new15',
 'new_F1520',
 'new_F1150',
 'new25',
 'MANAGERCOUNTINCOUNTRY_fixed',
 'new_F2110',
 'new_F2210',
 'F2340',
 'new11',
 'new_F2330',
 'new2',
 'F1210',
 'F2200_LAG1',
 'new8',
 'new_F1230','new_double']

In [ ]:
all_tune_df_last = all_tune_df[top30_test].drop(['OKVED_CODE_fixed','WORKERSRANGE'],axis=1)

In [ ]:
last_num = ['new16','new_F2200','new_F2300','new_F2400','F2410','new1','AGE','SIGNEDNUMBER_95_EVER','new15','new_F1520','new_F1150','new25','new_F2110','new_F2210','F2340','new11','new_F2330','new2','F1210','F2200_LAG1','new8','new_F1230']

In [ ]:
#all_tune_df.grcoupby('WORKERSRANGE').aggregate('mean').reset_index()

In [ ]:
t = 0
for x in last_num:
    print(t)
    all_tune_df_last[x] = all_tune_df_last[[x]].fillna(all_tune_df_last.groupby('new_double').transform(mean_nan))
    t+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


In [ ]:
fimoirtall_tune_df_last['new_double']

0              RARE_VALUE6 .. 10
1              RARE_VALUE6 .. 10
2               RARE_VALUE0 .. 5
3        79701000001.0251 .. 500
4               79701000001.0nan
                  ...           
13678        45323000000.00 .. 5
13679        45327000000.00 .. 5
13680        45329000000.00 .. 5
13681    45321000000.0101 .. 150
13682        45321000000.00 .. 5
Name: new_double, Length: 13683, dtype: object

In [ ]:
all_tune_df_last

,id_client,id_contract,OKTMO_CODE_fixed,new16,OKATO_REGIONCODE_y,new_F2200,TELEPHONECOUNT_fixed,new_F2300,new_F2400,OKFS_GROUP,OKATO_FED_y,F2410,new1,AGE,SIGNEDNUMBER_95_EVER,new15,new_F1520,new_F1150,new25,MANAGERCOUNTINCOUNTRY_fixed,new_F2110,new_F2210,F2340,new11,new_F2330,new2,F1210,F2200_LAG1,new8,new_F1230,new_double
0,1,5166.0,RARE_VALUE,-0.367981,79.0,-0.064579,1.0,0.028137,0.052724,Частная собственность,40.0,-0.372465,0.126109,NaN,-0.505679,2.201889,-0.511012,-0.286706,-0.665809,1.0,-0.368548,-0.297132,-0.311036,0.006726,0.165013,0.029042,-0.324632,-0.27778,0.051053,-0.291573,RARE_VALUE6 .. 10
1,2,2389.0,RARE_VALUE,-0.367981,79.0,-0.032348,2.0,0.056254,0.064745,Частная собственность,40.0,-0.409362,-0.388132,NaN,-0.505679,2.425569,-0.511012,-0.286706,-0.665809,1.0,-0.368548,-0.297132,-0.311036,0.006726,0.042460,-0.494423,-0.348328,-0.27778,0.051053,-0.126129,RARE_VALUE6 .. 10
2,3,17329.0,RARE_VALUE,-0.367981,79.0,-0.118916,2.0,-0.003946,0.012138,Частная собственность,40.0,-0.426207,-0.198237,NaN,-0.505679,-0.159313,-0.511012,-0.286706,-0.030974,1.0,-0.313843,-0.259581,-0.315285,-0.200325,0.059545,-0.494423,-0.348328,-0.27778,-0.080610,-0.571415,RARE_VALUE0 .. 5
3,5,15086.0,79701000001.0,-0.367981,79.0,2.538741,2.0,1.426763,1.282236,Частная собственность,40.0,0.720154,-0.388132,NaN,2.070937,-0.445877,-0.511012,-0.286706,-0.665809,1.0,-0.368548,-0.188024,-0.315285,0.968317,NaN,-0.494423,-0.348328,-0.27778,0.972098,-0.347558,79701000001.0251 .. 500
4,7,933.0,79701000001.0,NaN,79.0,NaN,NaN,NaN,NaN,NaN,40.0,NaN,NaN,1.102231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.407411,NaN,NaN,NaN,NaN,79701000001.0nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13678,12725,20931.0,45323000000.0,4.636739,45.0,0.050453,1.0,-0.010091,-0.002802,Частная собственность,30.0,-0.407579,-0.388132,NaN,-0.505679,-0.229595,-0.511012,-0.286706,-0.665809,1.0,-0.368548,NaN,-0.315285,-0.643650,NaN,1.053308,-0.348328,-0.27778,-0.636645,-0.374862,45323000000.00 .. 5
13679,12726,21787.0,45327000000.0,2.735644,45.0,-0.132091,RARE_VALUE,-0.345014,-0.312924,Частная собственность,30.0,-0.415054,-0.388132,NaN,-0.505679,1.642029,-0.499864,NaN,-0.665809,1.0,-0.368548,-0.297831,-0.315285,-0.039162,0.091285,-0.494423,-0.348328,-0.27778,-0.033367,-0.474435,45327000000.00 .. 5
13680,12728,20031.0,45329000000.0,1.097930,45.0,NaN,0.0,NaN,NaN,Частная собственность,30.0,-0.337215,NaN,NaN,0.782629,-0.445877,NaN,NaN,0.418701,4.0,NaN,NaN,-0.315285,NaN,NaN,NaN,-0.348328,NaN,NaN,NaN,45329000000.00 .. 5
13681,12729,24210.5,45321000000.0,NaN,45.0,NaN,RARE_VALUE,NaN,NaN,Частная собственность,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45321000000.0101 .. 150


In [ ]:
all_tune_df_last_train = all_tune_df_last.iloc[:8922,:]
all_tune_df_last_test = all_tune_df_last.iloc[8922:,:]

In [ ]:
all_tune_df_last_train['TARGET'] = labels_tune.apply(n05)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,id_client,id_contract,OKTMO_CODE_fixed,new16,OKATO_REGIONCODE_y,new_F2200,TELEPHONECOUNT_fixed,new_F2300,new_F2400,OKFS_GROUP,OKATO_FED_y,F2410,new1,AGE,SIGNEDNUMBER_95_EVER,new15,new_F1520,new_F1150,new25,MANAGERCOUNTINCOUNTRY_fixed,new_F2110,new_F2210,F2340,new11,new_F2330,new2,F1210,F2200_LAG1,new8,new_F1230,new_double
8922,4,19084.0,79701000001.0,NaN,79.0,-0.409447,1.0,-0.163187,-0.151764,Частная собственность,40.0,-0.425821,NaN,NaN,NaN,NaN,-0.511012,-0.286706,NaN,3.0,-0.368548,-0.255291,-0.315285,NaN,NaN,NaN,-0.348328,-0.27778,NaN,-0.455846,79701000001.051 .. 100
8923,6,24955.0,79701000001.0,-0.367981,79.0,-0.126977,2.0,-0.003511,0.018428,Государственная собственность,40.0,-0.415534,-0.388132,NaN,2.070937,-0.445877,-0.511012,-0.286706,-0.665809,2.0,-0.368548,NaN,NaN,0.968317,NaN,-0.494423,-0.348328,-0.27778,0.972098,-0.466136,79701000001.016 .. 50
8924,8,21717.0,79701000001.0,NaN,79.0,-0.174034,1.0,-0.001980,-0.007040,Государственная собственность,40.0,-0.051579,NaN,NaN,NaN,NaN,0.314339,-0.286706,NaN,1.0,-0.368548,-0.255291,-0.315285,NaN,NaN,NaN,-0.348328,-0.27778,NaN,-0.197423,79701000001.051 .. 100
8925,18,19618.5,79701000001.0,-0.367981,79.0,-0.248809,1.0,-0.061152,-0.055055,Частная собственность,40.0,-0.468821,NaN,NaN,2.070937,-0.445877,-0.511012,-0.286706,-0.665809,1.0,-0.368548,-0.184142,-0.315285,NaN,NaN,NaN,-0.348328,-0.27778,NaN,-0.432415,79701000001.00 .. 5
8926,19,18780.0,79701000001.0,NaN,79.0,-0.094618,1.0,-0.039220,-0.022185,Частная собственность,40.0,NaN,NaN,NaN,NaN,NaN,-0.511012,-0.286706,NaN,1.0,-0.368548,-0.270371,-0.315285,NaN,NaN,NaN,-0.348328,-0.27778,NaN,-0.729150,79701000001.06 .. 10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13678,12725,20931.0,45323000000.0,4.636739,45.0,0.050453,1.0,-0.010091,-0.002802,Частная собственность,30.0,-0.407579,-0.388132,NaN,-0.505679,-0.229595,-0.511012,-0.286706,-0.665809,1.0,-0.368548,NaN,-0.315285,-0.643650,NaN,1.053308,-0.348328,-0.27778,-0.636645,-0.374862,45323000000.00 .. 5
13679,12726,21787.0,45327000000.0,2.735644,45.0,-0.132091,RARE_VALUE,-0.345014,-0.312924,Частная собственность,30.0,-0.415054,-0.388132,NaN,-0.505679,1.642029,-0.499864,NaN,-0.665809,1.0,-0.368548,-0.297831,-0.315285,-0.039162,0.091285,-0.494423,-0.348328,-0.27778,-0.033367,-0.474435,45327000000.00 .. 5
13680,12728,20031.0,45329000000.0,1.097930,45.0,NaN,0.0,NaN,NaN,Частная собственность,30.0,-0.337215,NaN,NaN,0.782629,-0.445877,NaN,NaN,0.418701,4.0,NaN,NaN,-0.315285,NaN,NaN,NaN,-0.348328,NaN,NaN,NaN,45329000000.00 .. 5
13681,12729,24210.5,45321000000.0,NaN,45.0,NaN,RARE_VALUE,NaN,NaN,Частная собственность,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45321000000.0101 .. 150


In [ ]:
N_THREADS = mp.cpu_count() # threads cnt for lgbm and linear models
N_FOLDS = 10 # folds cnt for AutoML - 10 is better
RANDOM_STATE = 42 # fixed random state for various reasons
TIMEOUT = 3600 # Time in seconds for automl run
TARGET_NAME = 'TARGET' # Target column name

In [ ]:
%%time

roles = {'target': TARGET_NAME,
         'drop': ['id_contract','id_client']
         }

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.06 µs


In [ ]:
%%time

task = Task('binary', loss='logloss', metric='auc')

CPU times: user 6.78 ms, sys: 3 µs, total: 6.79 ms
Wall time: 14.9 ms


In [ ]:
#### %%time 
automl = TabularAutoML(task = task, 
                       timeout = 2400,
                       cpu_limit = N_THREADS,
                       #general_params = {'use_algos': [['linear_l2', 'lgb', 'lgb_tuned']]},
                       reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE},
                      )
oof_pred = automl.fit_predict(all_tune_df_last_train, roles = roles)
logging.info('oof_pred:\n{}\nShape = {}'.format(oof_pred, oof_pred.shape))

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(all_tune_df_last_train.TARGET,oof_pred.data)

0.8041478353317297

In [ ]:
pred=automl.predict(all_tune_df_last_test)

In [ ]:
ot = pd.DataFrame()

In [ ]:
ot['k']=pred.data.squeeze()

In [ ]:
ot.to_csv('phone.csv')

In [ ]:
all_tune_df['new_double'] = all_tune_df[['OKTMO_CODE_fixed','WORKERSRANGE']].apply(lambda x:str(x['OKTMO_CODE_fixed'])+str(x['WORKERSRANGE']),axis=1)

In [ ]:
all_tune_df['new_F1150']

In [ ]:
def obb(x,y):
    return str(x)+str(y)

In [ ]:
all_tune_df[['OKTMO_CODE_fixed','WORKERSRANGE']]

,OKTMO_CODE_fixed,WORKERSRANGE
0,RARE_VALUE,6 .. 10
1,RARE_VALUE,6 .. 10
2,RARE_VALUE,0 .. 5
3,79701000001.0,251 .. 500
4,79701000001.0,NaN
...,...,...
13678,45323000000.0,0 .. 5
13679,45327000000.0,0 .. 5
13680,45329000000.0,0 .. 5
13681,45321000000.0,101 .. 150


,k
0,0.060064
1,0.019316
2,0.020526
3,0.074543
4,0.061556
...,...
4756,0.794643
4757,0.586295
4758,0.327800
4759,0.295420


In [ ]:
tar = []

In [ ]:
all_tune_df_last_train

,id_client,id_contract,OKTMO_CODE_fixed,new16,OKATO_REGIONCODE_y,new_F2200,TELEPHONECOUNT_fixed,new_F2300,new_F2400,OKFS_GROUP,OKATO_FED_y,F2410,new1,AGE,SIGNEDNUMBER_95_EVER,new15,new_F1520,new_F1150,new25,MANAGERCOUNTINCOUNTRY_fixed,new_F2110,new_F2210,F2340,new11,new_F2330,new2,F1210,F2200_LAG1,new8,new_F1230,new_double,TARGET
0,1,5166.0,RARE_VALUE,-0.367981,79.0,-0.064579,1.0,0.028137,0.052724,Частная собственность,40.0,-0.372465,0.126109,NaN,-0.505679,2.201889,-0.511012,-0.286706,-0.665809,1.0,-0.368548,-0.297132,-0.311036,0.006726,0.165013,0.029042,-0.324632,-0.277780,0.051053,-0.291573,RARE_VALUE6 .. 10,1
1,2,2389.0,RARE_VALUE,-0.367981,79.0,-0.032348,2.0,0.056254,0.064745,Частная собственность,40.0,-0.409362,-0.388132,NaN,-0.505679,2.425569,-0.511012,-0.286706,-0.665809,1.0,-0.368548,-0.297132,-0.311036,0.006726,0.042460,-0.494423,-0.348328,-0.277780,0.051053,-0.126129,RARE_VALUE6 .. 10,0
2,3,17329.0,RARE_VALUE,-0.367981,79.0,-0.118916,2.0,-0.003946,0.012138,Частная собственность,40.0,-0.426207,-0.198237,NaN,-0.505679,-0.159313,-0.511012,-0.286706,-0.030974,1.0,-0.313843,-0.259581,-0.315285,-0.200325,0.059545,-0.494423,-0.348328,-0.277780,-0.080610,-0.571415,RARE_VALUE0 .. 5,1
3,5,15086.0,79701000001.0,-0.367981,79.0,2.538741,2.0,1.426763,1.282236,Частная собственность,40.0,0.720154,-0.388132,NaN,2.070937,-0.445877,-0.511012,-0.286706,-0.665809,1.0,-0.368548,-0.188024,-0.315285,0.968317,NaN,-0.494423,-0.348328,-0.277780,0.972098,-0.347558,79701000001.0251 .. 500,0
4,7,933.0,79701000001.0,NaN,79.0,NaN,NaN,NaN,NaN,NaN,40.0,NaN,NaN,1.102231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.407411,NaN,NaN,NaN,NaN,79701000001.0nan,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8917,12731,10258.0,RARE_VALUE,-0.367981,55.0,-0.118916,3.0,-0.003946,0.012138,Частная собственность,NaN,-0.426207,-0.198237,NaN,2.070937,-0.445877,-0.320531,-0.278111,-0.665809,3.0,-0.313843,-0.259581,-0.295858,0.968317,0.059545,-0.069997,-0.321139,-0.277780,-1.038831,-0.284794,RARE_VALUE0 .. 5,1
8918,12732,10005.0,RARE_VALUE,-0.367981,55.0,-0.118916,3.0,-0.003946,0.012138,Частная собственность,NaN,-0.426207,-0.198237,NaN,-0.505679,-0.445877,-0.320531,-0.278111,-0.665809,3.0,-0.313843,-0.259581,-0.295858,-1.046642,0.059545,-0.069997,-0.321139,-0.277780,-1.038831,-0.284794,RARE_VALUE0 .. 5,0
8919,12733,5291.0,RARE_VALUE,-0.367981,55.0,-0.049334,2.0,0.035340,0.048906,Иностранная собственность,NaN,-0.279090,-0.158473,NaN,-0.505679,-0.445877,-0.281829,-0.213418,-0.665809,2.0,-0.252553,-0.331544,-0.311271,-1.046642,0.207996,2.465699,-0.217041,-0.214128,0.972098,-0.298767,RARE_VALUE11 .. 15,0
8920,12734,3304.0,RARE_VALUE,2.563841,55.0,-0.118916,1.0,-0.003946,0.012138,Частная собственность,NaN,-0.426207,-0.198237,NaN,-0.505679,-0.445877,-0.320531,-0.278111,-0.665809,2.0,-0.313843,-0.259581,-0.295858,-1.046642,0.059545,-0.069997,-0.321139,-0.277780,-0.080610,-0.284794,RARE_VALUE0 .. 5,1


In [ ]:
id = list(all_tune_df_last_train['id_client'].array)

In [ ]:
dec = dict(zip(id,labels_tune))

In [ ]:
id_test = list(all_tune_df_last_test['id_client'].array)

In [ ]:
test_df

,id_contract,id_client,IP_flag,F1100,F1110,F1150,F1160,F1170,F1180,F1190,F1200,F1210,F1220,F1230,F1240,F1250,F1260,F1300,F1310,F1320,F1350,F1360,F1370,F1400,F1410,F1420,F1450,F1500,F1510,F1520,F1530,F1550,F1600,F1700,F2100,F2110,F2120,F2200,F2210,F2220,...,new3,new4,new5,new6,new7,new8,new9,new10,new11,new12,new13,new14,new15,new16,new17,new18,new19,new20,new21,new22,new23,new24,new25,new26,new27,new28,new_F1150,new_F1230,new_F1410,new_F1510,new_F1520,new_F2110,new_F2120,new_F2200,new_F2210,new_F2220,new_F2300,new_F2320,new_F2330,new_F2400
17891,17892,3620,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17892,17893,4101,0,1906000.0,NaN,1906000.0,NaN,NaN,NaN,NaN,14838000.0,4392000.0,NaN,727000.0,NaN,9719000.0,NaN,15222000.0,150000.0,NaN,NaN,NaN,15072000.0,NaN,NaN,NaN,NaN,1522000.0,NaN,1522000.0,NaN,NaN,16744000.0,16744000.0,2998000.0,310151000.0,307153000.0,2998000.0,NaN,NaN,...,NaN,NaN,NaN,3.5,0.000000,0.425414,380500.553191,0.011844,0.498308,317635.836773,0.001661,0.711794,1.801756e+05,0.006738,0.726465,1.672409e+05,6.530387,inf,7.649351,11.340426,9.466808,8.504983,34.5,8.680280,9.256845,8.592306,-816000.0,-1312000.0,NaN,NaN,-13679000.0,-16752000.0,-15658000.0,-1094000.0,NaN,NaN,-1094000.0,NaN,NaN,-876000.0
17893,17894,9589,0,147000.0,NaN,147000.0,NaN,NaN,NaN,NaN,95529000.0,29812000.0,7000.0,56538000.0,504000.0,8430000.0,238000.0,4145000.0,2538000.0,NaN,NaN,NaN,1607000.0,NaN,NaN,NaN,NaN,91531000.0,NaN,91531000.0,NaN,NaN,95676000.0,95676000.0,48975000.0,506023000.0,457048000.0,1753000.0,47222000.0,NaN,...,124509.285714,inf,7.0,10.3,0.064516,0.451613,334651.818182,0.135593,0.412429,731514.666667,0.277778,0.194444,3.393157e+05,0.065445,0.473822,2.859378e+05,5.709677,12.0,5.214286,5.727273,12.519233,21.222222,5.0,51.714286,49.142857,41.412171,-952000.0,-66171000.0,NaN,NaN,-75373000.0,355987000.0,339989000.0,466000.0,15532000.0,NaN,749000.0,NaN,NaN,683000.0
17894,17895,11546,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1906000.0,304000.0,NaN,682000.0,NaN,920000.0,NaN,1038000.0,NaN,NaN,NaN,NaN,NaN,95000.0,95000.0,NaN,NaN,774000.0,NaN,774000.0,NaN,NaN,1907000.0,1907000.0,NaN,9793000.0,8731000.0,1062000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.000000,0.500000,NaN,0.000000,0.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,561000.0,0.0,NaN,468000.0,8315000.0,7378000.0,937000.0,NaN,NaN,914000.0,NaN,NaN,858000.0
17895,17896,12558,0,26000.0,NaN,NaN,NaN,26000.0,NaN,NaN,2735000.0,1324000.0,NaN,1410000.0,NaN,1000.0,NaN,1220000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1541000.0,179000.0,1362000.0,NaN,NaN,2761000.0,2761000.0,NaN,6407000.0,6301000.0,106000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,623000.0,NaN,59000.0,11000.0,355000.0,802000.0,-447000.0,NaN,NaN,-445000.0,NaN,NaN,-356000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25216,25217,8128,0,1395000.0,NaN,1395000.0,NaN,NaN,NaN,NaN,515000.0,163000.0,NaN,318000.0,NaN,34000.0,NaN,44000.0,NaN,NaN,NaN,NaN,NaN,726000.0,NaN,NaN,726000.0,1140000.0,300000.0,840000.0,NaN,NaN,1910000.0,1910000.0,NaN,1518000.0,1424000.0,94000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,308000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25217,25218,1132,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [ ]:
test_pred = list(ot['k'].array)

In [ ]:
#test_pred = list(test_pred)

In [ ]:
for i in range(len(id_test)):
    if id_test[i] in id:
        tar.append(dec[id_test[i]])
    else:
        tar.append(test_pred[i])

In [ ]:
len(tar)

4761

In [ ]:
otpr_df['TARGET'] = tar

In [ ]:
len(tar)

4761

In [ ]:
dec_test = dict(zip(list(all_tune_df_last_test['id_client'].array),tar))

In [ ]:
last_test_df = all_df.loc[17891:,['id_client','id_contract']]

In [ ]:
last_tar = []
last_id = list(last_test_df['id_client'].array)

In [ ]:
for x in last_id:
    last_tar.append(dec_test[x])

In [ ]:
last_test_df['TARGET'] = last_tar

In [ ]:
last_test_df = last_test_df.drop(['id_client'],axis=1)

In [ ]:
a = pd.read_csv('/content/trying_ouh3aaa.csv',sep=';')

In [ ]:
last_test_df['TARGET'] = 0.3*last_test_df['TARGET']+0.7*a.TARGET.array

In [ ]:
last_test_df['TARGET']

17891    0.017388
17892    0.037606
17893    0.081729
17894    0.185982
17895    0.745230
           ...   
25216    0.236186
25217    0.013118
25218    0.315224
25219    0.367876
25220    0.029276
Name: TARGET, Length: 7330, dtype: float64

In [ ]:
last_test_df.to_csv('trying_mean_2f2.csv', index=False,sep=';')

In [ ]:
train_tune_df

,id_client,id_contract,F1100,F1110,F1150,F1160,F1170,F1180,F1190,F1200,F1210,F1220,F1230,F1240,F1250,F1260,F1300,F1310,F1320,F1350,F1360,F1370,F1400,F1410,F1420,F1450,F1500,F1510,F1520,F1530,F1550,F1600,F1700,F2100,F2110,F2120,F2200,F2210,F2220,F2300,...,new22,new23,new24,new25,new26,new27,new28,new_F1150,new_F1230,new_F1410,new_F1510,new_F1520,new_F2110,new_F2120,new_F2200,new_F2210,new_F2220,new_F2300,new_F2320,new_F2330,new_F2400,IP_flag_y,EGRPOINCLUDED_y,OKFS_GROUP,OKOPF_GROUP,OKOGU_GROUP,OKATO_REGIONCODE_y,OKATO_FED_y,OKTMO_CODE_fixed,OKTMO_FED_y,WORKERSRANGE,OKVED_CODE_fixed,FLAG_DISQUALIFICATION_y,SEX_NAME,CITIZENSHIP_NAME,TELEPHONECOUNT_fixed,MANAGERCOUNTINCOUNTRY_fixed,MANAGERCOUNTINREGION_fixed,MANAGERINNCOUNT_fixed,TARGET
0,1,5166.0,-0.237203,NaN,-0.247628,NaN,NaN,NaN,NaN,-0.327317,NaN,NaN,NaN,NaN,-0.265977,NaN,-0.250692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.311619,NaN,-0.33284,NaN,NaN,-0.27275,-0.272697,NaN,-0.313297,-0.327784,-0.277209,NaN,NaN,-0.271915,...,NaN,NaN,-0.307364,-0.665809,-0.219946,-0.257933,-0.14951,-0.286706,NaN,NaN,NaN,-0.511012,-0.368548,-0.386088,-0.064579,NaN,NaN,0.028137,NaN,NaN,0.052724,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,6 .. 10,43.11,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1
1,2,2389.0,-0.237203,NaN,-0.247628,NaN,-0.250965,NaN,NaN,-0.327317,-0.348328,NaN,-0.319713,NaN,-0.265977,NaN,-0.250692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.311619,-0.269537,-0.33284,NaN,NaN,-0.27275,-0.272697,NaN,-0.313297,-0.327784,-0.277209,NaN,NaN,-0.271915,...,NaN,NaN,-0.307364,-0.665809,-0.219946,-0.257933,-0.14951,-0.286706,-0.126129,NaN,0.118212,-0.511012,-0.368548,-0.386088,-0.032348,NaN,NaN,0.056254,NaN,0.04246,0.064745,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,6 .. 10,31.09,NaN,NaN,NaN,2.0,1.0,1.0,1.0,0
2,3,17329.0,-0.237203,NaN,-0.247628,NaN,NaN,NaN,NaN,-0.327317,-0.348328,-0.387576,-0.319713,NaN,-0.265977,-0.34809,-0.250692,-0.245664,NaN,NaN,NaN,NaN,-0.233301,-0.243715,NaN,NaN,-0.311619,NaN,-0.33284,NaN,NaN,-0.27275,-0.272697,-0.279925,-0.313297,-0.327784,-0.277209,-0.298098,-0.396543,-0.271915,...,NaN,NaN,-0.307364,NaN,NaN,NaN,NaN,-0.286706,-0.571415,-0.196372,NaN,-0.511012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,0 .. 5,10.11,NaN,NaN,NaN,2.0,1.0,1.0,1.0,1
3,5,15086.0,-0.237203,NaN,-0.247628,NaN,NaN,-0.390905,-0.323928,-0.327317,-0.348328,NaN,-0.319713,-0.326928,-0.265977,NaN,-0.250692,-0.194176,NaN,NaN,NaN,-0.25464,NaN,NaN,NaN,NaN,-0.311619,NaN,-0.33284,NaN,-0.35222,-0.27275,-0.272697,-0.279925,-0.313297,-0.327784,0.031521,-0.298098,NaN,-0.271915,...,-0.371822,-0.366503,-0.307364,-0.665809,-0.219946,-0.257933,-0.14951,-0.286706,-0.347558,NaN,NaN,-0.511012,-0.368548,-0.386088,2.538741,-0.188024,NaN,1.426763,0.013589,NaN,1.282236,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,79701000001.0,40.0,251 .. 500,10.51,NaN,NaN,NaN,2.0,1.0,1.0,1.0,0
4,7,933.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,79.0,40.0,79701000001.0,40.0,NaN,47.52.7,NaN,мужской,Российская Федерация,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8917,12731,10258.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [ ]:
all_tune_df

,id_client,id_contract,F1100,F1110,F1150,F1160,F1170,F1180,F1190,F1200,F1210,F1220,F1230,F1240,F1250,F1260,F1300,F1310,F1320,F1350,F1360,F1370,F1400,F1410,F1420,F1450,F1500,F1510,F1520,F1530,F1550,F1600,F1700,F2100,F2110,F2120,F2200,F2210,F2220,F2300,...,new22,new23,new24,new25,new26,new27,new28,new_F1150,new_F1230,new_F1410,new_F1510,new_F1520,new_F2110,new_F2120,new_F2200,new_F2210,new_F2220,new_F2300,new_F2320,new_F2330,new_F2400,IP_flag_y,EGRPOINCLUDED_y,OKFS_GROUP,OKOPF_GROUP,OKOGU_GROUP,OKATO_REGIONCODE_y,OKATO_FED_y,OKTMO_CODE_fixed,OKTMO_FED_y,WORKERSRANGE,OKVED_CODE_fixed,FLAG_DISQUALIFICATION_y,SEX_NAME,CITIZENSHIP_NAME,TELEPHONECOUNT_fixed,MANAGERCOUNTINCOUNTRY_fixed,MANAGERCOUNTINREGION_fixed,MANAGERINNCOUNT_fixed,new_double
0,1,5166.0,-0.237203,NaN,-0.247628,NaN,NaN,NaN,NaN,-0.327317,NaN,NaN,NaN,NaN,-0.265977,NaN,-0.250692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.311619,NaN,-0.33284,NaN,NaN,-0.27275,-0.272697,NaN,-0.313297,-0.327784,-0.277209,NaN,NaN,-0.271915,...,NaN,NaN,-0.307364,-0.665809,-0.219946,-0.257933,-0.14951,-0.286706,NaN,NaN,NaN,-0.511012,-0.368548,-0.386088,-0.064579,NaN,NaN,0.028137,NaN,NaN,0.052724,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,6 .. 10,43.11,NaN,NaN,NaN,1.0,1.0,1.0,1.0,RARE_VALUE6 .. 10
1,2,2389.0,-0.237203,NaN,-0.247628,NaN,-0.250965,NaN,NaN,-0.327317,-0.348328,NaN,-0.319713,NaN,-0.265977,NaN,-0.250692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.311619,-0.269537,-0.33284,NaN,NaN,-0.27275,-0.272697,NaN,-0.313297,-0.327784,-0.277209,NaN,NaN,-0.271915,...,NaN,NaN,-0.307364,-0.665809,-0.219946,-0.257933,-0.14951,-0.286706,-0.126129,NaN,0.118212,-0.511012,-0.368548,-0.386088,-0.032348,NaN,NaN,0.056254,NaN,0.04246,0.064745,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,6 .. 10,31.09,NaN,NaN,NaN,2.0,1.0,1.0,1.0,RARE_VALUE6 .. 10
2,3,17329.0,-0.237203,NaN,-0.247628,NaN,NaN,NaN,NaN,-0.327317,-0.348328,-0.387576,-0.319713,NaN,-0.265977,-0.34809,-0.250692,-0.245664,NaN,NaN,NaN,NaN,-0.233301,-0.243715,NaN,NaN,-0.311619,NaN,-0.33284,NaN,NaN,-0.27275,-0.272697,-0.279925,-0.313297,-0.327784,-0.277209,-0.298098,-0.396543,-0.271915,...,NaN,NaN,-0.307364,NaN,NaN,NaN,NaN,-0.286706,-0.571415,-0.196372,NaN,-0.511012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,0 .. 5,10.11,NaN,NaN,NaN,2.0,1.0,1.0,1.0,RARE_VALUE0 .. 5
3,5,15086.0,-0.237203,NaN,-0.247628,NaN,NaN,-0.390905,-0.323928,-0.327317,-0.348328,NaN,-0.319713,-0.326928,-0.265977,NaN,-0.250692,-0.194176,NaN,NaN,NaN,-0.254640,NaN,NaN,NaN,NaN,-0.311619,NaN,-0.33284,NaN,-0.35222,-0.27275,-0.272697,-0.279925,-0.313297,-0.327784,0.031521,-0.298098,NaN,-0.271915,...,-0.371822,-0.366503,-0.307364,-0.665809,-0.219946,-0.257933,-0.14951,-0.286706,-0.347558,NaN,NaN,-0.511012,-0.368548,-0.386088,2.538741,-0.188024,NaN,1.426763,0.013589,NaN,1.282236,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,79701000001.0,40.0,251 .. 500,10.51,NaN,NaN,NaN,2.0,1.0,1.0,1.0,79701000001.0251 .. 500
4,7,933.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,79.0,40.0,79701000001.0,40.0,NaN,47.52.7,NaN,мужской,Российская Федерация,NaN,NaN,NaN,NaN,79701000001.0nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [ ]:
a = pd.read_csv('/content/trying_ouh3aaa.csv',sep=';')

In [ ]:
all_tar = list(labels_tune.array)+list(a['TARGET'].apply(n05).array)

In [ ]:
all_tune_df['TARGET'] = all_tar

ValueError: ignored

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
a1 = pd.read_csv('/content/trying_kruto0999__2021-11-09T00_21_58.207288 03_00-2.csv',sep=';')
a2 = pd.read_csv('/content/trying_mean_2f2__2021-11-15T17_49_13.392986 03_00.csv',sep=';')
a3 = pd.read_csv('/content/trying_ouh3b__2021-11-10T02_08_58.581867 03_00.csv',sep=';')
a4 = pd.read_csv('/content/trying_scaled blend3__2021-11-15T00_08_50.096534 03_00.csv',sep=';')

In [ ]:
tar = (a1.TARGET+a2.TARGET+a3.TARGET+a4.TARGET)/4

In [ ]:
a1['TARGET'] = list(tar.array)

In [ ]:
a1.TARGET

0       0.011392
1       0.038022
2       0.099895
3       0.216368
4       0.876968
          ...   
7325    0.298488
7326    0.007697
7327    0.362437
7328    0.422045
7329    0.039196
Name: TARGET, Length: 7330, dtype: float64

In [ ]:
a1.to_csv('trying_super_blend.csv',sep=';',index=False)

In [ ]:
a1

,id_contract,TARGET
0,17892,0.011392
1,17893,0.038022
2,17894,0.099895
3,17895,0.216368
4,17896,0.876968
...,...,...
7325,25217,0.298488
7326,25218,0.007697
7327,25219,0.362437
7328,25220,0.422045
